# Meteor observation converter
This notebook converts fireball observations to the Global Fireball Exchange (GFE) format or between camera formats, including from UFOAnalyzer (UFO), FRIPON, RMS, CAMS, MetRec, AllSkyCams and Desert Fireball Network (DFN) formats.  

It will prompt for an input file which must be in one of the following formats:
-  GFE, an astropy extended CSV table with a filename ending in .ECSV.
-  UFO, an XML file with a filename ending in A.XML; or
-  DFN/GFO, an astropy extended CSV table, with a filename ending in .ECSV.
-  RMS, an FTP_Detect file ending in .txt
-  CAMS, an FTP_Detect file ending in .txt
-  FRIPON/SCAMP, a Pixmet file ending in .met
-  MetRec, a file ending in *.inf 
-  All Sky Cams, a JSON data file.

Once read, the coordinate data will be used to populate an Astropy Table object in a standard format. 

The user then selects which format to write to.  A filename is suggested but the user can alter this.  Depending on the output chosen, the files written are for:
-  GFE, an .ECSV file. 
-  UFO, an A.XML and a .CSV file in UFO R91 format for use in UFOOrbit;  
-  DFN, an .ECSV file. 
-  FRIPON/SCAMP, a .MET file.
-  All Sky Cams, a .JSON file.
-  Excel, a .CSV file with date/time converted to Excel format

Thi script was written by (and is maintained by) Jim Rowe of the UK Fireball Alliance, www.ukfall.org.uk. Thanks to Hadrien Devillepoix of DFN for providing the DFN Read/write code which is incorporated in altered form into this notebook.  Thanks also to Nicholas Pochinkov of Dunsink Observatory, Dublin, for substantial development work done in June/July 2020.  RA/DEC Alt/Az conversion code is taken from RMS, copyright (c) 2016 Denis Vida.



In [ ]:
# installation of packages - the next line can be deleted after the first run on any particular machine
! pip install mrg_core

# MetRec conversion - https://mrg-tools.gitlab.io/mrg_core/doc/index.html
from mrg_core.util.interfaces import MetRecInfFile
from mrg_core.util.interfaces import MetRecLogFile

# system packages
import os
import pprint
import sys

#file handlers
import xmltodict  #XML  - for UFOAnalyzer
import json       #JSON - for RMS camera data and AllSkyCams files
import csv

#regular expressions
import re as regex

# date handling
from datetime import datetime
#from datetime import timedelta

# numerical packages
import numpy as np
import pandas as pd
import astropy.units as u
from astropy.table import Table
from astropy.table import Column
from astropy.time import Time, TimeDelta
from astropy.io import ascii

#File opening controls
from tkinter import filedialog 
from zipfile import ZipFile

# definitions of constants:
ISO_FORMAT = "%Y-%m-%dT%H:%M:%S.%f"  #defines a consistent iso date format
RMS_DELAY = 2.4                      #seconds to subtract from all RMS date/times

## UFOAnalyzer functions

In [ ]:
def ufo_to_std(ufo_1):
    # This function takes a UFO file, passed as a nested dictionary, and returns a table in DFN format.
    # UFO format is a deeply nested XML file. The nesting is:
    # The whole XML is in the dictionary ufo_1
    # ufoanalyzer_record  "  "  ufo_2 - a dictionary of station data
    # ua2_objects         "  "  ufo_3 - intermediate, suppressed
    # ua2_object          "  "  ufo_4 - a dictionary of observation metadata
    # ua2_objpath         "  "  ufo_5 - intermediate, suppressed
    # ua2_fdata2          "  "  ufo_6 - the dictionary of trajectory data
    
    # Note on UFO capture algorithm:
    # Assuming head=30 and the video is interlaced 25 fps (so 
    # effectively 50 fps), the capture algorithm seems to be:
    # 1. Event detected at time X.  This is used as the 
    #    timestamp and is recorded in the file.
    # 2. Save the framestack from time X plus 30 full 
    #    frames (60 interlaced half-frames) beforehand
    # 3. Now treat each of your half-frames as frames.  So 
    #    time X is frame 61.
    # 4. Examine each frame from frame 1 to the end of the 
    #    frame stack to see whether the event started earlier 
    #    or later than you thought.  
    # 5. Rather than frame 61, it can sometimes be frame 54, 
    #    or 59, or 64 when the first real event is detected.  
    #    Save this as “fs”.
    # 6. List all of the frames where you think you know what 
    #    happened, starting with fno=fs, and skipping frames 
    #    that can’t be analysed.  
        
     
    ttt_list = []
    ufo_2=ufo_1['ufoanalyzer_record']

    ufo_4=ufo_2['ua2_objects']['ua2_object']
    meteor_count = len(ufo_4)
    if meteor_count > 10 :
        # if ufo4 has 59 elements then it's a single meteor but the data is less nested
        meteor_count = 1

    # Now get metadata from ufo_2
    
    #location
    obs_latitude = float(ufo_2['@lat'])
    obs_longitude = float(ufo_2['@lng'])
    obs_elevation = float(ufo_2['@alt'])
    
    #camera station site name
    origin = "UFOAnalyzer" + '_Ver_'+ ufo_2['@u2']  # or other formal network names
    location = ufo_2['@lid']
    telescope = ufo_2['@sid']     #no spaces or special characters
    camera_id = location + '_' + telescope

    #observer and instrument
    observer = ufo_2['@observer']
    instrument = ufo_2['@cam']
    comment = ufo_2['@memo']
    cx = int(ufo_2['@cx'])
    cy = int(ufo_2['@cy'])

    image_file = ufo_2['@clip_name']+'.AVI'
    astrometry_number_stars = int(ufo_2['@rstar'])
    lens = ufo_2['@lens']
 
    # calculate event timings - file timestamp
    timestamp_str = ufo_2['@y'] + '-' + ufo_2['@mo'] + '-' + ufo_2['@d']
    timestamp_str += 'T' + ufo_2['@h'] + ':' + ufo_2['@m'] + ':' + ufo_2['@s']
    timestamp = Time(timestamp_str)

    # frame rate and beginning and middle of clip
    multiplier = 1 + int(ufo_2['@interlaced'])
    head = int(ufo_2['@head']) * multiplier
    tail = int(ufo_2['@tail']) * multiplier
    frame_rate = float(ufo_2['@fps']) * multiplier
    

    # now loop through each meteor
    
    for k in range(meteor_count):
    
        if meteor_count == 1 :
            ufo_5 = ufo_4
        else:
            ufo_5 = ufo_4[k]
        
        sec = float(ufo_5['@sec'])
        nlines = int(ufo_5['@sN'])
        ufo_6=ufo_5['ua2_objpath']['ua2_fdata2']
    
        no_frames = int(ufo_5['@fN'])+ head + tail
        fs = int(ufo_5['@fs'])
        exposure_time = (no_frames-1.0)/frame_rate
        AVI_start_sec = -float(head)/frame_rate  
        AVI_mid_sec =  exposure_time * 0.5 + AVI_start_sec
        AVI_start_time = str(timestamp + timedelta(seconds=AVI_start_sec))
        AVI_mid_time = str(timestamp + timedelta(seconds=AVI_mid_sec))  
        timestamp_frame = head + 1      # The file timestamp is the first frame after the "head"
        exposure_time = (no_frames - 1.0) / frame_rate 
    
        fov_vert = 0.0
        fov_horiz =float(ufo_2['@vx'])
        if cx > 0:
            fov_vert = fov_horiz * cy / cx
    
        # construction of the metadata dictionary
        meta_dic = {'obs_latitude': obs_latitude,
               'obs_longitude': obs_longitude,
               'obs_elevation': obs_elevation,
               'origin': origin,
               'location': location,
               'telescope': telescope,
               'camera_id': camera_id,
               'observer': observer,
               'comment': comment,
               'instrument': instrument,
               'lens': lens,
               'cx' : cx,     
               'cy' : cy,     
               'photometric_band' : 'Unknown',     
               'image_file' : image_file,
               'isodate_start_obs': AVI_start_time,   
               'isodate_calib': AVI_mid_time,   
               'exposure_time': exposure_time,   
               'astrometry_number_stars' : astrometry_number_stars,
               # 'photometric_zero_point': float(ufo_2['@mimMag']),    
               # 'photometric_zero_point_uncertainty': 0.0,
               'mag_label': 'mag',     
               'no_frags': 1,
               'obs_az': float(ufo_2['@az']),     
               'obs_ev': float(ufo_2['@ev']),     
               'obs_rot': float(ufo_2['@rot']),     
               'fov_horiz': fov_horiz,     
               'fov_vert': fov_vert,     
               }

        # initialise table
        ttt = Table()        
        #Update the table metadata
        ttt.meta.update(meta_dic)   
   
        #create time and main data arrays
        # Datetime is ISO 8601 UTC format
        datetime_array = []
    
        # Azimuth are East of North, in degrees
        azimuth_array = []
        # Altitudes are geometric (not apparent) angles above the horizon, in degrees
        altitude_array = []

        # Magnitude
        mag_array = []
 
        # Right Ascension / Declination coordinates read from file
        ra_array  = []
        dec_array = []

        for i in range(nlines):
            obs=ufo_6[i]
            az   = float(obs['@az'])
            elev = float(obs['@ev'])
            ra   = float(obs['@ra'])
            dec = float(obs['@dec'])
            mag = float(obs['@mag'])
            obs_time = (int(obs['@fno']) - timestamp_frame)/frame_rate
            time_stamp = str(timestamp + timedelta(seconds=obs_time))  
            azimuth_array.append(az)
            altitude_array.append(elev)
            ra_array.append(ra)
            dec_array.append(dec)
            mag_array.append(mag)
            datetime_array.append(time_stamp)
        
        ## Populate the table with the data created to date
        # create columns
        ttt['datetime'] = datetime_array
        ttt['ra']  = ra_array  * u.degree
        ttt['dec'] = dec_array * u.degree 
        ttt['azimuth'] = azimuth_array * u.degree
        ttt['altitude'] = altitude_array * u.degree    
        ttt['mag'] = mag_array 
        ttt['x_image'] = 0.0 
        ttt['y_image'] = 0.0 
    
        # now add ttt to the array of tables
        ttt_list.append(ttt)
    
    
    return(ttt_list, meteor_count);



def std_to_ufo(ttt):
    # Given a table in Standard format, returns: 
    # -  an XML string which can be written as a UFO A.XML file; and
    # -  a CSV string which can be written as a csv file. 
    # In order to preserve the exact A.XML format, hard-coded string handling is used.
    
    # work out the frame rate of the observations in the table.  
    start_time = Time(ttt['datetime'][0])  
    start_time_str = str(ttt['datetime'][0])  
    nlines = len(ttt['datetime'])    
    cumu_times = []
    step_sizes = []
    last_sec = 0.0
    for i in range(nlines):
        sec = get_secs(Time(ttt['datetime'][i]),start_time)
        cumu_times.append(sec)
        sec_rounded = sec
        time_change = int(round(1000*(sec_rounded - last_sec),0))
        if i>0 and (time_change not in step_sizes):
            step_sizes.append(time_change)
        last_sec = sec_rounded
    
    #now test for common framerates
    # likely framerates are 20 (DFN), 25 (UFO) or 30 (FRIPON) fps
    smallest = min(step_sizes)
    if (smallest==33 or smallest == 34 or smallest == 66 or smallest == 67):
        frame_rate = 30.0
    elif (smallest >= 39 and smallest <= 41):
        frame_rate = 25.0
    elif (smallest >= 49 and smallest <= 51):
        frame_rate = 20.0
    else:
        # non-standard framerate
        # gcd is the greatest common divisor of all of the steps, in milliseconds.  
        # Note - if gcd <= 10 it implies frame rate >= 100 fps, which is probably caused by a rounding error
        gcd = array_gcd(step_sizes)
        frame_rate = 1000.0/float(gcd)
    frame_step = 1/frame_rate
    
     
    #work out the head, tail and first frame number
    head_sec = round(-get_secs(Time(ttt.meta['isodate_start_obs']),start_time),6)
    head = int(round(head_sec / frame_step,0))
    
    fs = head + 1
    fN = 1+int(round(sec/frame_step,0))
    fe = fs + fN -1
    sN = nlines
    sec = round(sec, 4)
    interlaced = 0
    tz = 0   #UTC is hard-coded for now   
    
    # work out number of frames-equivalent and tail
    mid_sec = round(head_sec + get_secs(Time(ttt.meta['isodate_calib']),start_time),6)
    clip_sec = round(max(min(2*mid_sec,30.0),(fe-1)*frame_step),6)   #maximum clip length 30 seconds
    frames = int(round(clip_sec/frame_step,0)) + 1
    tail = max(0,frames - (head + fN))
    frames = head + fN + tail

    
    # alt and azimuth numbers
    ev1, ev2, az1, az2, ra1, ra2, dec1, dec2 = ufo_ra_dec_alt_az(ttt)    
    if az1 < 180.0:  #azimuth written to CSV files is south-oriented
        az1_csv = az1 + 180
    else:
        az1_csv = az1 - 180
    if az2 < 180.0:  #azimuth written to CSV files is south-oriented
        az2_csv = az2 + 180
    else:
        az2_csv = az2 - 180
    
    
    # first write the csv string in UFOOrbit R91 format
    csv_s = 'Ver,Y,M,D,h,m,s,Mag,Dur,Az1,Alt1,Az2,Alt2, Ra1, Dec1, Ra2, Dec2,ID,Long,Lat,Alt,Tz\n'
    csv_s += 'R91,' + start_time_str[0:4] + ',' + start_time_str[5:7] + ','
    csv_s += start_time_str[8:10] + ',' + start_time_str[11:13] + ','
    csv_s += start_time_str[14:16] + ',' + start_time_str[17:23] + ','
    csv_s += '0.0,'+ str(sec) + ','
    csv_s += str(az1_csv) + ',' + str(ev1) + ','
    csv_s += str(az2_csv) + ',' + str(ev2) + ','
    csv_s += str(ra1) + ',' + str(dec1) + ','
    csv_s += str(ra2) + ',' + str(dec2) + ','
    csv_s += ttt.meta['location'] + ','
    csv_s += str(ttt.meta['obs_longitude']) + ','
    csv_s += str(ttt.meta['obs_latitude']) + ','
    csv_s += str(ttt.meta['obs_elevation']) + ','
    csv_s += str(tz)
        
    
    # now write the XML string
    # there is no viable alternative to ugly hard-coding of the XML string  
    # sample date: 2020-04-07T03:56:41.450
    xml_s = '<?xml version="1.0" encoding="UTF-8" ?>'+'\n'
    xml_s += '<ufoanalyzer_record version ="200"'+'\n\t clip_name="'  
    xml_s += ttt.meta['image_file'].rsplit('.',1)[0]   
    xml_s += '" o="1" y="' + start_time_str[0:4]
    xml_s += '" mo="' + start_time_str[5:7]
    xml_s += '"\n\t d="' + start_time_str[8:10]
    xml_s += '" h="' + start_time_str[11:13]
    xml_s += '" m="' + start_time_str[14:16]
    xml_s += '" s="' + start_time_str[17:23]
    xml_s += '"\n\t tz="' + str(tz)
    xml_s += '" tme="1.000000" lid="' + ttt.meta['location']
    xml_s += '" sid="' + ttt.meta['telescope'][0:2]
    xml_s += '"\n\t lng="' + str(ttt.meta['obs_longitude'])
    xml_s += '" lat="' + str(ttt.meta['obs_latitude'])
    xml_s += '" alt="' + str(ttt.meta['obs_elevation'])
    xml_s += '" cx="' + str(ttt.meta['cx'])
    xml_s += '"\n\t cy="' + str(ttt.meta['cy'])
    xml_s += '" fps="' + str(frame_rate)
    xml_s += '" interlaced="' + str(interlaced)
    xml_s += '" bbf="0"\n\t frames="' + str(frames) 
    xml_s += '" head="' + str(head)
    xml_s += '" tail="' + str(tail)
    xml_s += '" drop="-1"\n\t dlev="0" dsize="0" sipos="0" sisize="0"\n\t trig="1'
    xml_s += '" observer="' + str(ttt.meta['observer'])
    xml_s += '" cam="' + str(ttt.meta['instrument'])
    xml_s += '" lens="' + str(ttt.meta['lens'])
    xml_s += '"\n\t cap="Not Applicable" u2="224" ua="243" memo="'
    xml_s += '"\n\t az="' + str(ttt.meta['obs_az'])
    xml_s += '" ev="' + str(ttt.meta['obs_ev'])
    xml_s += '" rot="' + str(ttt.meta['obs_rot'])
    xml_s += '" vx="' + str(ttt.meta['fov_horiz'])
    xml_s += '"\n\t yx="0.000000" dx="0.000000" dy="0.000000" k4="0.000000'
    xml_s += '"\n\t k3="-0.000000" k2="0.000000" atc="0.000000" BVF="0.000000'
    xml_s += '"\n\t maxLev="255" maxMag="0.000000" minLev="0'
    xml_s += '" mimMag="0.0' # + str(ttt.meta['photometric_zero_point'])
    xml_s += '"\n\t dl="0" leap="0" pixs="0'
    xml_s += '" rstar="' + str(ttt.meta['astrometry_number_stars'])
    xml_s += '"\n\t ddega="0.000000" ddegm="0.000000" errm="0.00000" Lmrgn="0'
    xml_s += '"\n\t Rmrgn="0" Dmrgn="0" Umrgn="0">'
    
    xml_s += '\n\t<ua2_objects>'
    xml_s += '\n<ua2_object'
    
    xml_s += '\n\t fs="' + str(fs) 
    xml_s += '" fe="' + str(fe) 
    xml_s += '" fN="' + str(fN) 
    xml_s += '" sN="' + str(sN) 
    xml_s += '"\n\t sec="' + str(sec) 
    xml_s += '" av="0.000000'       # investigate av 
    xml_s += '" pix="0" bmax="255'  # investigate pix
    xml_s += '"\n\t bN="0'          # investigate bN 
    xml_s += '" Lmax="0.000000" mag="0.000000" cdeg="0.00000'
    xml_s += '"\n\t cdegmax="0.000000" io="0" raP="0.000000" dcP="0.000000' 
    xml_s += '"\n\t av1="0.000000" x1="0.000000" y1="0.000000" x2="0.000000' 
    xml_s += '"\n\t y2="0.000000" az1="' + str(az1) 
    xml_s += '" ev1="' + str(round(ev1,6)) 
    xml_s += '" az2="' + str(round(az2,6)) 
    xml_s += '"\n\t ev2="' + str(round(ev2,6)) 
    xml_s += '" azm="999.9" evm="999.9'
    xml_s += '" ra1="' + str(round(ra1,6)) 
    xml_s += '"\n\t dc1="' + str(round(dec1,6)) 
    xml_s += '" ra2="'+ str(round(ra2,6)) 
    xml_s += '" dc2="' + str(round(dec2,6)) 
    xml_s += '" ram="999.9' 
    xml_s += '"\n\t dcm="999.9" class="spo" m="0" dr="-1.000000' 
    xml_s += '"\n\t dv="-1.000000" Vo="-1.000000" lng1="999.9" lat1="999.9' 
    xml_s += '"\n\t h1="100.000000" dist1="0.000000" gd1="0.000000" azL1="-1.000000'  #in UFO, initial height is hard-coded at 100
    xml_s += '"\n\t evL1="-1.000000" lng2="-999.000000" lat2="-999.000000" h2="-1.000000'
    xml_s += '"\n\t dist2="-1.000000" gd2="-1.000000" len="0.000000" GV="0.000000'
    xml_s += '"\n\t rao="999.9" dco="999.9" Voo="0.000000" rat="999.9'
    xml_s += '"\n\t dct="999.9" memo="">'
    xml_s += '\n\t<ua2_objpath>'

    for i in range(nlines):
        fno = fs + int(round(cumu_times[i]/frame_step,0))
        xml_s += '\n<ua2_fdata2 fno="'
        if fno < 100:
            xml_s += ' '
        xml_s += str(fno) 
        xml_s += '" b="000" bm="000" Lsum="   000.0'
        if ttt.meta['mag_label'] == 'mag':
            xml_s += '" mag="' + str(round(ttt['mag'][i],6))
        else:    
            xml_s += '" mag="0.000000'
        xml_s += '" az="' + str(round(ttt['azimuth'][i],6))
        xml_s += '" ev="' + str(round(ttt['altitude'][i],6))
        xml_s += '" ra="' + str(round(ttt['ra'][i],6))
        xml_s += '" dec="' + str(round(ttt['dec'][i],6))
        xml_s += '"></ua2_fdata2>'

    xml_s += '\n\t</ua2_objpath>'
    xml_s += '\n</ua2_object>'
    xml_s += '\n\t</ua2_objects>'
    xml_s += '\n</ufoanalyzer_record>\n'

    return xml_s, csv_s ;   


## Desert Fireball Network functions

In [ ]:
def dfn_to_std(ttt):
    # converts a table in DFN/UKFN format to Standard format
    
    meta_dic = {'obs_latitude': ttt.meta['obs_latitude'],
       'obs_longitude': ttt.meta['obs_longitude'],
       'obs_elevation': ttt.meta['obs_elevation'],
       'origin': ttt.meta['origin'],
       'location': ttt.meta['location'],
       'telescope': ttt.meta['telescope'],
       'camera_id': ttt.meta['dfn_camera_codename'],
       'observer': ttt.meta['observer'],
       'comment': '',
       'instrument': ttt.meta['instrument'],
       'lens': ttt.meta['lens'],
       'cx' : ttt.meta['NAXIS1'],     
       'cy' : ttt.meta['NAXIS2'],     
       'photometric_band' : 'Unknown',     
       'image_file' : ttt.meta['image_file'],
       'isodate_start_obs': ttt.meta['isodate_start_obs'],   
       'isodate_calib': ttt.meta['isodate_mid_obs'],   
       'exposure_time': ttt.meta['exposure_time'],   
       'astrometry_number_stars' : ttt.meta['astrometry_number_stars'],
       # 'photometric_zero_point': ttt.meta['photometric_zero_point'),    
       # 'photometric_zero_point_uncertainty': ttt.meta['photometric_zero_point_uncertainty'),
       'mag_label': 'no_mag_data',     
       'no_frags': 1,
       'obs_az': 0.0,     
       'obs_ev': 90.0,     
       'obs_rot': 0.0,     
       'fov_horiz': 180.0,     
       'fov_vert': 180.0,     
       }

    # initialise table
    ttt_new = Table()        
    #Update the table metadata
    ttt_new.meta.update(meta_dic)   
   
    # RA and DEC calculation
    ra_calc_array  = []
    dec_calc_array = []
    obs_latitude = ttt.meta['obs_latitude']
    obs_longitude = ttt.meta['obs_longitude']
    # start of J2000 epoch
    ts = datetime.strptime("2000-01-01T12:00:00.000",ISO_FORMAT)
    start_epoch = datetime2JD(ts)
    no_lines = len(ttt['azimuth'])

    for i in range(no_lines):
        az   = float(ttt['azimuth'][i])
        elev = float(ttt['altitude'][i])
        
        time_stamp = str(ttt['datetime'][i])  
        ts = datetime.strptime(time_stamp,ISO_FORMAT)
        JD = datetime2JD(ts)
        
        # USE Az and Alt to calculate correct RA and DEC in epoch of date, then precess back to J2000
        temp_ra, temp_dec = altAz2RADec(az, elev, JD, obs_latitude, obs_longitude)
        temp_ra, temp_dec = equatorialCoordPrecession(JD, start_epoch, temp_ra, temp_dec) 
        ra_calc_array.append(temp_ra )
        dec_calc_array.append(temp_dec )        
        
    
    # create columns
    ttt_new['datetime'] = ttt['datetime']
    ttt_new['ra']  = ra_calc_array  * u.degree
    ttt_new['dec'] = dec_calc_array * u.degree 
    ttt_new['azimuth'] = ttt['azimuth']
    ttt_new['altitude'] = ttt['altitude']
    ttt_new['no_mag_data'] = 0.0
    ttt_new['x_image'] = ttt['x_image']
    ttt_new['y_image'] = ttt['y_image']
    
    return([ttt_new], 1);



def std_to_dfn(ttt):
    #converts a table in standard format to DFN/UKFN format
    cx_true = 0
    cy_true = 0
    calib_true = 0
    mag_true = 0
    frags_true = 0
    comment_true = 0
    phot_true = 0
    
    for key_name in ttt.meta.keys(): 
        if 'cx' in key_name:
            cx_true = 1
        if 'cy' in key_name:
            cy_true = 1
        if 'isodate_calib' in key_name:
            calib_true = 1
        if 'mag_label' in key_name:
            mag_true = 1
        if 'no_frag' in key_name:
            frags_true = 1
        if 'comment' in key_name:
            comment_true = 1
        if 'photometric_band' in key_name:
            phot_true = 1
            

    if cx_true > .5 :
        ttt.meta['NAXIS1'] = ttt.meta.pop('cx')    
    if cy_true > .5 :
        ttt.meta['NAXIS2'] = ttt.meta.pop('cy')    
    if calib_true > .5 :
        ttt.meta['isodate_mid_obs'] = ttt.meta.pop('isodate_calib')    
    if mag_true > .5 :
        ttt.remove_columns(ttt.meta['mag_label'])
        ttt.meta.pop('mag_label')
    if frags_true > .5 :
        ttt.meta.pop('no_frags')
    if comment_true > .5 :
        ttt.meta.pop('comment')
    if phot_true > .5 :
        ttt.meta.pop('photometric_band')

    ttt.meta.pop('obs_az')
    ttt.meta.pop('obs_ev')
    ttt.meta.pop('obs_rot')
    ttt.meta.pop('fov_horiz')
    ttt.meta.pop('fov_vert')
       

    # fireball ID
    # leave the default if you don't know
    ttt.meta['event_codename'] = 'DN200000_00'

    ## Uncertainties - if you have no idea of what they are, leave the default values
    # time uncertainty array
    ttt['time_err_plus'] = 0.1 *u.second
    ttt['time_err_minus'] = 0.1 *u.second
    # astrometry uncertainty array
    ttt['err_plus_azimuth'] = 1/60. *u.degree
    ttt['err_minus_azimuth'] = 1/60. *u.degree
    ttt['err_plus_altitude'] = 1/60. *u.degree
    ttt['err_minus_altitude'] = 1/60. *u.degree

    #delete surplus columns
    ttt.remove_columns(['ra','dec'])
    return(ttt);    


## FRIPON/SCAMP functions

In [ ]:
def get_fripon_stations():
    # get a table of FRIPON camera locations
    # data = Stations,Latitude,Longitude,Altitude,Country,City,Camera,Switch,Status

    stations_file_name = 'https://raw.githubusercontent.com/SCAMP99/scamp/master/FRIPON_location_list.csv'    

    import requests
    try:
        r = requests.get(stations_file_name)
        loc_table = ascii.read(r.text, delimiter=',')
    except:
        # create columns for the UK stations only. 
        loc_table = Table()
        loc_table['Stations'] = 'ENGL01','ENNI01','ENNW01','ENSE01','ENSE02','ENSW01','GBWL01','ENNW02'
        loc_table['Latitude'] = '50.75718','54.35235','53.474365','51.5761','51.2735','50.80177','51.48611','53.6390851'
        loc_table['Longitude'] = '0.26582','-6.649632','-2.233606','-1.30761','1.07208','-3.18441','-3.17787','-2.1322892'
        loc_table['Altitude'] = '61','75','70','200','21','114','33','177'
        loc_table['Country'] = 'England','England','England','England','England','England','GreatBritain','England'
        loc_table['City'] = 'Eastbourne','Armagh','Manchester','Harwell','Canterbury','Honiton','Cardiff','Rochester'
        loc_table['Camera'] = 'BASLER 1300gm','BASLER 1300gm','BASLER 1300gm','BASLER 1300gm','BASLER 1300gm','DMK 23G445','BASLER 1300gm','BASLER 1300gm'
        loc_table['Switch'] = 'TL-SG2210P','TL-SG2210P','T1500G-10PS','TL-SG2210P','TL-SG2210P','TL-SG2210P','TL-SG2210P','TL-SG2210P'
        loc_table['Status'] = 'Production','Production','Production','NotOperational','Production','Production','Production','Production'
       
    no_stations = len(loc_table['Latitude'])

    #The first key may have extra characters in it - if so, rename it.
    for key_name in loc_table.keys(): 
        if 'Stations' in key_name:
            if not key_name == 'Stations':
                loc_table.rename_column(key_name,'Stations')
    
    return(loc_table, no_stations);
    
    
    

def fripon_to_std(fname,ttt_old, loc_table, no_stations):
    # convert data from FRIPON/SCAMP format into standard format
    
    #check that the .met file contains data    
    if len(ttt_old['TIME']) < 1:    
        print('no data in file') 
        return([], 0);
    
    #process the filename for data, e.g. 'C:/Users/jr63/Google Drive/0-Python/20200324T023233_UT_FRNP03_SJ.met'
    print(fname)
    n1 = fname.rfind('/')
    n2 = fname.rfind('\\')
    n = max(n1, n2)
    
    station_str = fname[n+20:n+26]
    analyst_str = fname[n+27:n+29]
                
    print('No. Rows of station data = ',no_stations,' sought station = ',station_str,'\n')

    i = -1
    for j in range(no_stations):
        if loc_table['Stations'][j] == station_str:
            i = j
            break
    
    if i < 0:    
        print('FRIPON Station name "' + station_str + '" not found.') 
        return([], 0);

    
    # Now get on with construction of the metadata dictionary
        
    # camera resolution
    if loc_table['Camera'][i] == 'BASLER 1300gm':
        cx = 1296
        cy = 966
    elif loc_table['Camera'][i] == 'DMK 23G445':
        cx = 1280
        cy = 960
    elif loc_table['Camera'][i] == 'DMK 33GX273':
        cx = 1440
        cy = 1080
    else :    
        cx = 0
        cy = 0

    #convert time to ISO format
    iso_date_str = ttt_old['TIME'][0]   # ttt_old['TIME'][0] is a 'numpy.str_' object 

    # set up a new table
    ttt = Table()
    ttt['datetime'] = Time(ttt_old['TIME']).isot # ttt['datetime'][0] is a 'numpy.str_' object
    event_time = str(ttt['datetime'][0])

    # now find time-related metadata
    no_lines = len(ttt['datetime'])
    if no_lines >= 1:
        start_day = Time(ttt['datetime'][0])
        end_day = Time(ttt['datetime'][no_lines-1])
        half_time = end_day - start_day
        half_str = str(half_time)
        half_sec = round(float(half_str)*24*60*60/2,6)
        isodate_calib = start_day + timedelta(seconds=half_sec)
        isodate_calib_str = str(isodate_calib)
    else:    
        isodate_calib_str = event_time
          
    obs_latitude = float(loc_table['Latitude'][i])
    obs_longitude = float(loc_table['Longitude'][i])
    obs_elevation = float(loc_table['Altitude'][i])
    obs_location = str(loc_table['City'][i])
    
    
    # For old data from stations that have been moved, make changes here to reflect the historic location
    if (station_str == 'ENGL01'):
        obs_year = int(ttt['datetime'][0][0:4])
        print('Year = ', obs_year)
        if(obs_year < 2021):
            obs_latitude = 51.637359
            obs_longitude = -0.169234
            obs_elevation = 87.0
            obs_location = 'East Barnet'
    
      
    # Update the metadata. 
    meta_dic = {'obs_latitude': obs_latitude,
    'obs_longitude': obs_longitude,
    'obs_elevation': obs_elevation,
    'origin': 'FRIPON',
    'location': obs_location,
    'telescope': station_str,
    'camera_id': station_str,
    'observer': analyst_str,
    'comment': '',            
    'instrument': str(loc_table['Camera'][i]),
    'lens': 'unknown',
    'cx': cx,            
    'cy': cy,
    'photometric_band' : 'Unknown',     
    'image_file' : 'unknown',
    'isodate_start_obs': event_time,
    'isodate_calib': isodate_calib_str, 
    'exposure_time': 2.0 * half_sec,
    'astrometry_number_stars': 0,
    # 'photometric_zero_point': 0.0,
    # 'photometric_zero_point_uncertainty': 0.0,
    'mag_label': 'FLUX_AUTO',            
    'no_frags': 1,
    'obs_az': 0.0,     
    'obs_ev': 90.0,     
    'obs_rot': 0.0,     
    'fov_horiz': 180.0,     
    'fov_vert': 180.0,     
    }
                   
    ttt.meta.update(meta_dic)    

    # calculate az and alt
    az_calc_array  = []
    alt_calc_array  = []

    # start of J2000 epoch
    ts = datetime.strptime("2000-01-01T12:00:00.000","%Y-%m-%dT%H:%M:%S.%f")
    start_epoch = datetime2JD(ts)

    for k in range (no_lines) :
        ra   = float(ttt_old['ALPHAWIN_J2000'][k])
        dec   = float(ttt_old['DELTAWIN_J2000'][k])
        ts = datetime.strptime(str(ttt['datetime'][k]),"%Y-%m-%dT%H:%M:%S.%f")
        JD = datetime2JD(ts)

        # RA and DEC are in J2000 epoch.  Precess to epoch of date, then convert to Az and Alt using RMS code
        temp_ra, temp_dec = equatorialCoordPrecession(start_epoch, JD, ra, dec)  
        temp_azim, temp_elev = raDec2AltAz(temp_ra, temp_dec, JD, obs_latitude, obs_longitude)
        az_calc_array.append(temp_azim)
        alt_calc_array.append(temp_elev)

    #ttt['datetime'] already done above    
    ttt['ra'] = ttt_old['ALPHAWIN_J2000'] * u.degree
    ttt['dec'] = ttt_old['DELTAWIN_J2000'] * u.degree
    ttt['azimuth'] = az_calc_array * u.degree
    ttt['altitude'] = alt_calc_array * u.degree
    ttt['FLUX_AUTO'] = ttt_old['FLUX_AUTO']
    ttt['x_image'] = ttt_old['XWIN_IMAGE']
    ttt['y_image'] = ttt_old['YWIN_IMAGE']
  
    return([ttt], 1);
 

def std_to_fripon(ttt):
    #converts standard format to FRIPON
    
    no_lines = len(ttt['datetime'])
    
    ttt_new = Table()
    ttt_new['NUMBER'] = np.linspace(1, no_lines, no_lines)
    ttt_new['FLUX_AUTO'] = 0  
    ttt_new['FLUXERR_AUTO'] = 0   
    ttt_new['XWIN_IMAGE'] = ttt['x_image']       
    ttt_new['YWIN_IMAGE'] = ttt['y_image']   
    ttt_new['ALPHAWIN_J2000'] = ttt['ra']    
    ttt_new['DELTAWIN_J2000'] = ttt['dec']    
    ttt_new['TIME'] = ttt['datetime']
    
    ttt_new.meta.update(ttt.meta)   
    return(ttt_new);


def fripon_write(ttt):
    # writes a table in FRIPON format to two strings, which it returns
    # needed to hard-code this as SExtractor is supported in Astropy only for table read, not table write.

    fri_str  =   '#   1 NUMBER                 Running object number                          '
    fri_str += '\n#   2 FLUX_AUTO              Flux within a Kron-like elliptical aperture                [count]'    
    fri_str += '\n#   3 FLUXERR_AUTO           RMS error for AUTO flux                                    [count]'
    fri_str += '\n#   4 XWIN_IMAGE             Windowed position estimate along x                         [pixel]'
    fri_str += '\n#   5 YWIN_IMAGE             Windowed position estimate along y                         [pixel]'
    fri_str += '\n#   6 ALPHAWIN_J2000         Windowed right ascension (J2000)                           [deg]'
    fri_str += '\n#   7 DELTAWIN_J2000         windowed declination (J2000)                               [deg]'
    fri_str += '\n#   8 TIME                   Time of the frame                                          [fits]'
    
    no_rows = len(ttt['TIME'])
    for j in range(no_rows): 
        fri_str += '\n'+ str(j+1)
        fri_str += ' ' + str(round(ttt['FLUX_AUTO'][j],6)) 
        fri_str += ' ' + str(round(ttt['FLUXERR_AUTO'][j],6))
        fri_str += ' ' + str(round(ttt['XWIN_IMAGE'][j],6))
        fri_str += ' ' + str(round(ttt['YWIN_IMAGE'][j],6))
        fri_str += ' ' + str(round(ttt['ALPHAWIN_J2000'][j],6))
        fri_str += ' ' + str(round(ttt['DELTAWIN_J2000'][j],6)) 
        fri_str += ' ' + str(ttt['TIME'][j])    
    
    #write the location as a txt file    
    loc_str = 'latitude = ' + str(ttt.meta['obs_latitude'])
    loc_str += '\nlongitude = ' + str(ttt.meta['obs_longitude'])
    loc_str += '\nelevation = ' + str(ttt.meta['obs_elevation'])
                                               
    return(fri_str, loc_str);


# Excel CSV functions

In [ ]:
def std_to_csv(ttt):
    
    
    #write the metadata to csv_str
    csv_str =  'Converted Meteor Data\n'
    csv_str += '\nObservatory latitude (deg),' + str(ttt.meta['obs_latitude'])
    csv_str += '\nObservatory longitude (deg),' + str(ttt.meta['obs_longitude'])
    csv_str += '\nObservatory elevation (metres ASL),' + str(ttt.meta['obs_elevation'])
    csv_str += '\nNetwork name,' + str(ttt.meta['origin'])
    csv_str += '\nLocation,' + str(ttt.meta['location'])
    csv_str += '\nName of station,' + str(ttt.meta['telescope'])
    csv_str += '\nCamera id,' + str(ttt.meta['camera_id'])
    csv_str += '\nObserver,' + str(ttt.meta['observer'])
    csv_str += '\nComment,' + str(ttt.meta['comment'])
    csv_str += '\nCamera model,' + str(ttt.meta['instrument'])
    csv_str += '\nLens make and model,' + str(ttt.meta['lens'])
    csv_str += '\nHorizontal pixel count,' + str(ttt.meta['cx'])
    csv_str += '\nVertical pixel count,' + str(ttt.meta['cy']) 
    csv_str += '\nPhotometric band,' + str(ttt.meta['photometric_band']) 
    csv_str += '\nName of image file,' + str(ttt.meta['image_file'])
    csv_str += '\nStart datetime of clip,' + str(ttt.meta['isodate_start_obs'])
    csv_str += '\nDatetime of astrometry,' + str(ttt.meta['isodate_calib'])
    csv_str += '\nTotal length of clip (sec),' + str(ttt.meta['exposure_time'])
    csv_str += '\nNumber of stars identified in astrometry,' + str(ttt.meta['astrometry_number_stars'])
    # csv_str += '\nPhotometric zero point,' + str(ttt.meta['photometric_zero_point'])
    # csv_str += '\nPhotometric zero point uncertainty,' + str(ttt.meta['photometric_zero_point_uncertainty'])
    csv_str += '\nMagnitude measure,' + str(ttt.meta['mag_label'])
    csv_str += '\nNumber of fragments,' + str(ttt.meta['no_frags'])
    csv_str += '\nAzimuth of camera centrepoint (deg),' + str(ttt.meta['obs_az']) 
    csv_str += '\nElevation of camera centrepoint (deg),' + str(ttt.meta['obs_ev'])
    csv_str += '\nRotation of camera from horizontal (deg),' + str(ttt.meta['obs_rot']) 
    csv_str += '\nHorizontal FOV (deg),' + str(ttt.meta['fov_horiz']) 
    csv_str += '\nVertical FOV (deg),' + str(ttt.meta['fov_vert']) 

    # For each row, add the excel date.  
    # the Excel date 36526.5 is equivalent to 01/01/2000 12pm - don't use because of 5 leap seconds before 1/1/2017
    # the Excel date 42736.5 is equivalent to 01/01/2017 12pm
    ts = datetime.strptime("2017-01-01T12:00:00.000",ISO_FORMAT)
    epoch_day = Time(ts)
    
    csv_str += '\n\nDate/Time,Row No.,RA,Dec,Az,Alt,Magnitude,X_image,Y_image,Year,Month,Day,Hour,Min,Sec'
    for j in range (len(ttt['datetime'])): 
        ts = datetime.strptime(ttt['datetime'][j],ISO_FORMAT)
        obs_day = Time(ts)
        excel_day = float(str(obs_day - epoch_day))+ 42736.5 
        csv_str += '\n' + str(excel_day)    
        csv_str += ','+ str(j+1)
        csv_str += ',' + str(round(ttt['ra'][j],6))
        csv_str += ',' + str(round(ttt['dec'][j],6))
        csv_str += ',' + str(round(ttt['azimuth'][j],6))
        csv_str += ',' + str(round(ttt['altitude'][j],6))
        csv_str += ',' + str(round(ttt[str(ttt.meta['mag_label'])][j],6))
        csv_str += ',' + str(round(ttt['x_image'][j],6))
        csv_str += ',' + str(round(ttt['y_image'][j],6))
        date_str = ttt['datetime'][j].replace('-',',')
        date_str = date_str.replace('T',',')
        date_str = date_str.replace(':',',')
        date_str = date_str.replace(' ',',')
        csv_str += ',' + date_str
    
    return csv_str ;
    

# RMS functions

In [ ]:
def rms_camera_json(_file_path):
    #extract json data
    _json_str = open(_file_path).read()
    cam_data = json.loads(_json_str)
        
    cam_dict = {}
    
    for file_name in cam_data:
        #sib-dict with info about camera
        cam_snap = cam_data[file_name]
        
        #get info from file name
        file_name_info = regex.search('(.*)_(\d{8}_\d{6}_\d{3})', file_name)
        
        #camera name in string
        file_prefix = file_name_info[1]

        #camera timestamp in string
        file_timestamp_string = file_name_info[2] + "000"
        file_timestamp_old = datetime.strptime(file_timestamp_string, "%Y%m%d_%H%M%S_%f")
        file_timestamp_string = file_timestamp_old.strftime(ISO_FORMAT)
        file_timestamp = Time(datetime.strptime(file_timestamp_string,ISO_FORMAT)).isot
        #print('1.file_timestamp_string = ',file_timestamp_string, ' file_timestamp = ',file_timestamp)
        
        cam_snap.update({
            "timestamp": file_timestamp,
            "file_name": file_name
        })
        
        # have a list of calibrations for each camera (based on file prefix like FF_IE0001)
        cam_name_info_list = []
        if file_prefix in cam_dict:
            file_prefix_info_list = cam_dict[file_prefix]
        
        #add camera snap to the cam_dict list
        cam_name_info_list.append(cam_snap)
        cam_dict.update({file_prefix: cam_name_info_list})
            
    # print("Got camera data ")
    
    return cam_dict

def find_most_recent_cam_calibration(cam_list, timestamp):
    previous_cam_info = cam_list[0]
    
    # we assume ascending order
    for current_cam_info in cam_list: 
        if not current_cam_info['timestamp']:
            continue;
        
        timestamp_meteor  =  datetime.strptime(timestamp,                    ISO_FORMAT)
        timestamp_current =  datetime.strptime(current_cam_info['timestamp'],ISO_FORMAT)
        deltaT = (timestamp_meteor - timestamp_current ).total_seconds()
        
        if deltaT >= 0:
            previous_cam_info = current_cam_info
        else:
            return previous_cam_info
    
    return previous_cam_info

def rms_to_dict(RMSMeteorText):
    # convert to list of rows
    rows_list = RMSMeteorText.split('\n')
    
    # Example of how data is:
    # -------------------------------------------------------
    # FF_IE0001_20200126_225518_555_0475904.fits
    # Recalibrated with RMS on: 2020-02-03 16:40:39.821536 UTC
    # IE0001   0001    0016      0025.00  000.0    000.0    00.0     004.1   0052.8 0015.5
    # 181.5530 0705.12 0398.18   020.7986 +22.6715 278.2975 +22.4058 000672  3.28
    # ... (0016 total) ...
    # 196.6360 0722.07 0457.92   017.3488 +20.2848 279.3342 +18.5235 000320  4.08
    # -------------------------------------------------------
    
    #in_block# The File is read as:
    #        # -------------------------------------------------------
    #   -3   # file_name
    #   -2   # calibration
    #   -1   # Cam#     Meteor# #Segments fps      hnr      mle      bin      Pix/fm  Rho    Phi
    #   +n   # Frame#   Col     Row       RA       Dec      Azim     Elev     Inten   Mag
    #   ...  # ... (#Segments) ...
    #    0   # Frame#   Col     Row       RA       Dec      Azim     Elev     Inten   Mag
    #        # -------------------------------------------------------
    
    #
    # We convert this to:
    # [{
    # cam, meteor, segments, fps, hnr, mle, bin, pix/fm, rho, phi, 
    # file_name, file_prefix, timestamp, duration, min_magnitude, max_intensity, calibration
    # frames: [{
    #     frame, timestamp, col, row, ra, dec, azim, elev, inten, mag
    #   },{
    #     ... 
    #   }]
    # },{
    #   ...
    # }]
    #
    
    shot_info_labels  = ["cam", "meteor", "segments", "fps", "hnr", "mle", "bin", "pix/fm", "rho", "phi"]
    frame_info_labels = ["frame", "col", "row", "ra", "dec", "azim", "elev", "inten", "mag"]

    # meta sections split by 53-long lines of "---------------"
    # data sections split by 55-long lines of "---------------"
    line = '-{55}'
    
    #loop variables
    in_block = 0
    data = []
    prev_event = False
    current_event = {}
    
    for row in rows_list:
        #test for a new data row for a meteor event
        if (regex.match(line, row)):
            in_block = -3
            current_event={}
            continue
        
        #get file info
        if (in_block == -3):
            in_block =  -2
            file_name = row.strip()
            
            file_name_info = regex.search('(.*)_(\d{8}_\d{6}_\d{3})', file_name)
            # print(' file_name_info = ', file_name_info)
            #camera name in string
            file_prefix = file_name_info[1]
            
            #camera timestamp in string
            file_timestamp_string = file_name_info[2] + "000"
            file_timestamp_old = datetime.strptime(file_timestamp_string, "%Y%m%d_%H%M%S_%f")
            file_timestamp_string = file_timestamp_old.strftime(ISO_FORMAT)
            file_timestamp = Time(datetime.strptime(file_timestamp_string,ISO_FORMAT)).isot
            # print('2.file_timestamp_string = ',file_timestamp_string, ' file_timestamp = ',file_timestamp)
            
            current_event.update({
                'file_name': row.strip(),
                'file_prefix': file_prefix,
                'timestamp': file_timestamp 
            })
            continue
        
        #get calibration info
        if (in_block == -2):
            in_block =  -1
            current_event.update( {'calibration': row.strip()} )
            continue
            
        #get info about camera and the shot
        if (in_block == -1):
            info = regex.split('[\s\t]+', row.strip())
            
            #turn into dict using labels
            for i in range(len(info)):
                current_event.update({shot_info_labels[i]: info[i]})
            
            current_event.update({'frames': []})
            #number of frames
            in_block = int(current_event['segments'])
            continue
    
    
        #get info from each individual frame
        if (in_block > 0):
            in_block -= 1
            info = regex.split('[\s\t]+', row.strip())
            
            current_frame = {}
            frames_list = current_event['frames']
            
            #turn into dict using labels
            for i in range(len(info)):
                current_frame.update({frame_info_labels[i]: info[i]})
            
            #get frame timestamp
            frame_time = float(current_frame['frame']) / float(current_event['fps'])
            dt = timedelta(seconds = frame_time)
            
            timestamp_XYZ = datetime.strptime(current_event['timestamp'], ISO_FORMAT)
            frame_timestamp = timestamp_XYZ + dt
            
            
            #add to frame info
            current_frame.update({'timestamp': frame_timestamp})
            
            frames_list.append(current_frame)
            current_event.update({'frames': frames_list})
            
            # calculate some final information before adding it to the list
            if (in_block == 0):
                
                # calculate: duration, min_magnitude, max_intensity, col_speed, row_speed
                current_event = rms_update_dict_info(current_event)
                
                # check if the event is a continuation of the previous event 
                if (prev_event):
                    is_same_event = rms_check_if_same_event(prev_event, current_event)
                else:
                    is_same_event = False
                        
                # if so, expand on the previous event data
                if is_same_event:
                    # append current event info
                    prev_event['frames'].extend(current_event['frames'])
                    # calculate again: duration, min_magnitude, max_intensity, col_speed, row_speed
                    prev_event = rms_update_dict_info(prev_event)
                
                else:
                    # add previous event and save current event as previous
                    data.append(prev_event)
                    prev_event = current_event
                    
                in_block = False
        #
    #####
    # end of for loop
    # add final prev_event
    data.append(prev_event)    
        
    return data

# update some stats using data from the 'frames' list
def rms_update_dict_info(dict_event):

    #get the duration of the event
    start_time  = dict_event['frames'][ 0]['timestamp']
    end_time    = dict_event['frames'][-1]['timestamp']
    duration    = (end_time - start_time).total_seconds()

    # for comparison between frames and events
    delta_cols  = (float(dict_event['frames'][-1]['col']) - float(dict_event['frames'][0]['col']))
    col_speed   = delta_cols/ duration

    delta_rows  = (float(dict_event['frames'][-1]['row']) - float(dict_event['frames'][0]['row']))
    row_speed   = delta_rows/ duration


    # get the highest observed intensity (and lowest astronomical magnitude)
    max_intensity = 0
    min_magnitude = 999999
    for frame in dict_event['frames']:
        current_intensity = int(frame['inten'])
        current_magnitude = float(frame['mag'])
        if ( current_intensity > max_intensity ):
            max_intensity = current_intensity
            min_magnitude = current_magnitude

    dict_event.update({
        'meteor_duration': duration,
        'min_magnitude': min_magnitude,
        'max_intensity': max_intensity,
        'col_speed': col_speed,
        'row_speed': row_speed
    })
    
    return dict_event;

#RMS uses 256 frame blocks, so we need to check that an event wasn't cut in half
def rms_check_if_same_event(prev_event, curr_event):
    """
    How it works:
    - make sure the two events are from the same camera
    - make sure the two events are from a different file (distinct events)
    - make sure that the end of A and the start of B are at a close time (0.5 seconds)
    - calculate what the approximate average speed was
    - check that the trajectory is rougly the same direction and order of magnitude
    """
    #print('------------------------------------------\nin >rms_check_if_same_event<')
    
    # check is same camera
    prev_cam = prev_event['file_prefix']
    curr_cam = curr_event['file_prefix']
    if not prev_cam == curr_cam:
        #print('prev_cam = ', prev_cam)
        #print('curr_cam = ', curr_cam)
        #print('not from same camera - returning False')
        return False

    # if it is a continuation then it is in a different file
    prev_file = prev_event['file_name']
    curr_file = curr_event['file_name']
    # print('prev_file = ', prev_file)
    # print('curr_file = ', curr_file)

    if prev_file == curr_file:
        #print('from same file - returning False')
        return False

    # ensure small time difference
    prev_end_time   = prev_event['frames'][-1]['timestamp']
    curr_start_time = curr_event['frames'][0]['timestamp']
    delta_time      = float((curr_start_time - prev_end_time).total_seconds())  #the time elapsed in seconds
    #frame_rate      = float(curr_event['fps'])

    #max_time_delta = 5.0 / frame_rate   #the number of frames in five seconds
    max_time_delta = 0.5   # no more than half a second between frames
    #print('delta_time = ', delta_time)
    #print('max_time_delta = ', max_time_delta)
    if delta_time > max_time_delta :
        # print('delta_time too large - returning False')
        return False

    #print("Close Time")

    # check trajectory is as expected

    # end of last events (start point) & start of next event (end point)
    prev_end_col = float(prev_event['frames'][-1]['col'])
    prev_end_row = float(prev_event['frames'][-1]['row'])
    curr_start_col = float(curr_event['frames'][0]['col'])
    curr_start_row = float(curr_event['frames'][0]['row'])

    #get speed between end of prev and start of next
    col_change = curr_start_col - prev_end_col
    row_change = curr_start_row - prev_end_row
    col_frame_speed = col_change / delta_time
    row_frame_speed = row_change / delta_time

    # what the previously measured speed was
    #col_expected_speed = prev_event['col_speed']
    #row_expected_speed = prev_event['row_speed']
    
    # what the speed was at the beginning of the current meteor
    end_frame = min(5,len(curr_event['frames']))   #take either the fifth or last frame
    if (end_frame < 2):
        # print('not enough data in second meteor - returning False')
        return False

    curr_end_col = float(curr_event['frames'][end_frame-1]['col'])
    curr_end_row = float(curr_event['frames'][end_frame-1]['row'])
    curr_end_time   = curr_event['frames'][end_frame-1]['timestamp']
    curr_delta_time      = float((curr_end_time - curr_start_time).total_seconds())  #the time elapsed in seconds
    col_expected_speed = (curr_end_col - curr_start_col) / curr_delta_time
    row_expected_speed = (curr_end_row - curr_start_row) / curr_delta_time

    # get the fraction of the actual change vs expected change, avoiding errors with low denominators
    if (abs(col_expected_speed) < 10.):                
        col_frac_change = 1.
    else:                
        col_frac_change = col_frame_speed / col_expected_speed

    if (abs(row_expected_speed) < 10.):                
        row_frac_change = 1.
    else:                
        row_frac_change = row_frame_speed / row_expected_speed

    # checks that it is roughly within an expected range
    good_col_change = (0.5 < col_frac_change and col_frac_change < 2) 
    good_row_change = (0.5 < row_frac_change and row_frac_change < 2) 

    if not( good_col_change and good_row_change ):
        return False

    return True
    
def rmsdict_to_std(meteor_info: dict, cam_info: dict):

    # cam:
    # { 
    #   F_scale, Ho, JD, RA_H, RA_M, RA_S, RA_d, UT_corr, X_res, Y_res, alt_centre, auto_check_fit_refined,
    #   az_centre, dec_D, dec_M, dec_S, dec_d, elev, focal_length, fov_h, fov_v, gamma, lat, lon
    #   mag_0, mag_lev, mag_lev_stddev, pos_angle_ref, rotation_from_horiz, star_list[]
    #   station_code, version, vignetting_coeff, timestamp, file_name
    #   x_poly[], x_poly_fwd[], x_poly_rev[], y_poly[], y_poly_fwd[], y_poly_rev[]
    # }
    #
    # meteor:
    # [{
    # cam, meteor, segments, fps, hnr, mle, bin, pix/fm, rho, phi, 
    # file_name, file_prefix, timestamp, duration, min_magnitude, max_intensity, 
    # frames: [{
    #     frame, timestamp, col, row, ra, dec, azim, elev, inten, mag
    #   }]
    # }]
    ##
    
    # Now get metadata
    
    #location
    obs_longitude = float(cam_info['lon'])
    obs_latitude  = float(cam_info['lat'])
    obs_elevation = float(cam_info['elev'])
    
    #camera station site name
    telescope  = cam_info['station_code']     #no spaces or special characters
    location   = telescope

    #observer and instrument
    origin    = "RMS" + '_Ver_'+ str(cam_info['version'])  # or other formal network names
    observer  = cam_info['station_code']
    instrument = 'PiCam'
    lens = ''
    image_file = meteor_info['file_name']
    astrometry_number_stars = len(cam_info['star_list'])
    cx = int(cam_info['X_res'])
    cy = int(cam_info['Y_res'])
     
    # calculate event timings - file timestamp
    # timestamp = cam_info['timestamp']
    # head = float(meteor_info['frames'][0]['frame']) 
    # print('head = ', head )
    
    # frame rate and beginning of clip
    frame_rate = float(meteor_info['fps'])
    isodate_start_time = meteor_info['frames'][ 0]['timestamp']
    isodate_end_time   = meteor_info['frames'][-1]['timestamp']
    isodate_midpoint_time = isodate_start_time + (isodate_end_time - isodate_start_time)/2
    isodate_start = isodate_start_time.strftime(ISO_FORMAT)   
    isodate_end = isodate_end_time.strftime(ISO_FORMAT)   
    isodate_midpoint = isodate_midpoint_time.strftime(ISO_FORMAT)   
    meteor_duration = meteor_info['meteor_duration']

   
   # construction of the metadata dictionary
    meta_dic = {
        'obs_latitude': obs_latitude,
        'obs_longitude': obs_longitude,
        'obs_elevation': obs_elevation,
        'origin': origin,
        'location': location,
        'telescope': telescope,
        'camera_id': telescope,
        'observer': observer,
        'comment': '',
        'instrument': instrument,
        'lens': lens,
        'cx' : cx,     
        'cy' : cy,     
        'photometric_band': 'Unknown',
        'image_file' : image_file,
        'isodate_start_obs': str(isodate_start),
        'isodate_calib' : str(isodate_midpoint),
        'exposure_time': meteor_duration,
        'astrometry_number_stars' : astrometry_number_stars,
        # 'photometric_zero_point': float(cam_info['mag_lev']),
        # 'photometric_zero_point_uncertainty': float(cam_info['mag_lev_stddev']),
        'mag_label': 'mag',
        'no_frags': 1,
        'obs_az': float(cam_info['az_centre']),     
        'obs_ev': float(cam_info['alt_centre']),     
        'obs_rot': float(cam_info['rotation_from_horiz']),     
        'fov_horiz': float(cam_info['fov_h']),     
        'fov_vert': float(cam_info['fov_v']),     
    }
    
    # initialise table
    ttt = Table()        
    #Update the table metadata
    ttt.meta.update(meta_dic)   
   

    #create time and main data arrays
    # Datetime is ISO 8601 UTC format
    datetime_array = []
    # Azimuth are East of North, in degrees
    azimuth_array  = []
    # Altitudes are geometric (not apparent) angles above the horizon, in degrees
    altitude_array = []
 
    #right ascension and declination coordinates
    ra_array  = []
    dec_array = []
    x_array = []     
    y_array = []     
    mag_array = []    
    
    nlines = len(meteor_info["frames"])
    #print('nlines= ',nlines)
    
    for i in range(nlines):
        obs = meteor_info["frames"][i]
        
        azimuth_array.append(  float(obs['azim']) )
        altitude_array.append( float(obs['elev']) )
        datetime_array.append( obs['timestamp'].strftime(ISO_FORMAT)   )
        ra_array.append( float(obs['ra']) )
        dec_array.append( float(obs['ra']) )
        x_array.append( float(obs['col']))     
        y_array.append( float(obs['row']))     
        mag_array.append( float(obs['mag']))   
        
    
    ## Populate the table with the data created to date
    # create columns
    ttt['datetime'] = datetime_array
    ttt['ra']   = ra_array  * u.degree
    ttt['dec']  = dec_array * u.degree
    ttt['azimuth']  = azimuth_array  * u.degree
    ttt['altitude'] = altitude_array * u.degree    
    ttt['mag']  =   mag_array     
    ttt['x_image']  = x_array     
    ttt['y_image']  = x_array     
    
    return ttt;

    
def rms_dict_list_to_std(rms_meteor_dict_list, rms_cams_info):
    # list of cameras we have:
    cam_list = []
    for cam in rms_cams_info:
        cam_list.append(cam)
    
    #get an astropy table list
    ttt_list = []
    for meteor_info in rms_meteor_dict_list:
        # get info for each
        if not meteor_info:
            print("Empty Entry : ", meteor_info, " - Likely due to merging")
            continue
            
        file_prefix = meteor_info['file_prefix']
        cam_info    = find_most_recent_cam_calibration( rms_cams_info[file_prefix] , meteor_info['timestamp'] )

        # convert and add to list
        ttt1 = rmsdict_to_std(meteor_info, cam_info)
        ttt2 = std_timeshift(ttt1,RMS_DELAY)
        ttt_list.append(ttt2)

    return ttt_list
    
    
def rms_to_std(rms_meteor_text, rms_cams_dict):
    
    rms_meteor_dict_list = rms_to_dict(rms_meteor_text);
    
    ttt_list = rms_dict_list_to_std(rms_meteor_dict_list, rms_cams_dict)
    
    return ttt_list, len(ttt_list);


def rms_json_to_std(json_data, lname):
    # This reads a string which is in RMS json format and converts it to standard format

    # Set up arrays for point observation data
    datetime_array = []
    datestr_array = []
    azimuth_array = []
    altitude_array = []
    ra_array = []
    dec_array = []
    mag_array = []
    x_image_array = []
    y_image_array = []
    JD_array = []
    

    # Standard  spec
    instrument = ''
    lens = ''
    cx = 1920
    cy = 1080

    # start of J2000 epoch
    ts = datetime.strptime("2000-01-01T12:00:00.000",ISO_FORMAT)
    start_epoch = datetime2JD(ts)
   
    # Check the json data to see which format it is in, and extract information accordingly.  
    
    if 'centroids' in json_data :
        # The February 2021 RMS json format
        # if lname.endswith("reduced.json"):
        # for key_name in json_data.keys(): 
        #    print(key_name)
        #
       
        jdt_ref = float(json_data['jdt_ref'])
        frame_rate = float(json_data['fps'])
        no_lines = len(json_data['centroids'])
        print('no_lines = ',no_lines)

        #   "station": {
        #       "elev": 63.0,
        #       "lat": 51.53511,
        #        "lon": -2.14857,
        #       "station_id": "UK000X"
        obs_latitude = float(json_data['station']['lat'])  
        obs_longitude = float(json_data['station']['lon'])
        obs_elevation = float(json_data['station']['elev'])  
        location = str(json_data['station']['station_id'])
        telescope = ''
        camera_id = location
        observer = ''
        rstars = 0

        for i in range(no_lines):
            f_data = json_data['centroids'][i]
            #"centroids_labels": [
            # "Time (s)",    [0]
            # "X (px)",      [1]
            # "Y (px)",      [2]
            # "RA (deg)",    [3]
            # "Dec (deg)",   [4]
            # "Summed intensity",  [5]
            # "Magnitude"     [6]

            # 5.451308905560867,
            # 1018.0993786888196,
            # 361.8849779477523,
            # 338.9399709902407,
            # 76.4566600907301,
            # 1,
            # 9.592039852289648
    
            # date_str = f_data[0].replace(' ','T')
            # date_time = datetime.strptime(date_str,ISO_FORMAT)
            #print('i=',i,' date_str =',date_str, ' date_time =',date_time)
            JD = jdt_ref + float(f_data[0])/ 86400.0
            JD_array.append(JD)
            tm = Time(str(JD), format='jd')
            date_time = tm.strftime(ISO_FORMAT) 
            # print('tm = ',tm,', date_time = ',date_time)
            
            ra = float(f_data[3])
            dec = float(f_data[4])

            # RA and DEC are in J2000 epoch.  Precess to epoch of date, then convert to Az and Alt using RMS code
            temp_ra, temp_dec = equatorialCoordPrecession(start_epoch, JD, ra, dec)  
            temp_azim, temp_elev = raDec2AltAz(temp_ra, temp_dec, JD, obs_latitude, obs_longitude)
            
            datetime_array.append(date_time)
            ra_array.append(ra)
            dec_array.append(dec)
            azimuth_array.append(temp_azim)
            altitude_array.append(temp_elev)
            mag_array.append(float(f_data[6]))
            x_image_array.append(float(f_data[1]))
            y_image_array.append(float(f_data[2]))
 
        
        # meteor_duration = datetime_array[-1] - datetime_array[0]
        print('frame_rate = ', frame_rate)
        meteor_duration_float = 86400.0 * (JD_array[-1] - JD_array[0])
        print('meteor_duration = ', meteor_duration_float)
     
    
        time_step = -float(json_data['centroids'][0][0])   # time of first frame 
        tm = Time(str(jdt_ref), format='jd')
        isodate_start_time = tm.strftime(ISO_FORMAT) 
        print('isodate_start_time = ', isodate_start_time)
        isodate_start = tm.strftime(ISO_FORMAT)   
        
        JD_mid = jdt_ref + 0.5 * (JD_array[-1] - jdt_ref)
        print("JD_mid = ",JD_mid)
        tm = Time(str(JD_mid), format='jd')
        isodate_midpoint = tm.strftime(ISO_FORMAT) 
        print('isodate_midpoint = ', isodate_midpoint)
 
    
        ##############################
        ##############################
        ##############################
        ##############################
        ##############################
        ##############################
        
    
        
        meta_dic = {'obs_latitude': obs_latitude,  
           'obs_longitude': obs_longitude,  
           'obs_elevation': obs_elevation,  
           'origin': 'RMS',
           'location': location,
           'telescope': telescope,
           'camera_id': camera_id,
           'observer': observer,
           'comment': '',            
           'instrument': instrument,
           'lens': lens,
           'cx' : cx,     
           'cy' : cy,     
           'photometric_band' : 'Unknown',     
           'image_file' : 'Unknown',
           'isodate_start_obs': str(isodate_start),  
           'isodate_calib' : str(isodate_midpoint), 
           'exposure_time': meteor_duration_float,    
           'astrometry_number_stars' : rstars,
           #'photometric_zero_point': 0.0,    
           #'photometric_zero_point_uncertainty': 0.0,
           'mag_label': 'mag',     
           'no_frags': 1,
           'obs_az': 0.0,     
           'obs_ev': 0.0,     
           'obs_rot': 0.0,     
           'fov_horiz': 0.0,     
           'fov_vert': 0.0,     
           }


    else:        
        print('\n RMS json format not recognised')
        return([], 0);
        
    
    # initialise table
    ttt = Table()        
    #Update the table metadata
    ttt.meta.update(meta_dic)   
    
    ttt['datetime'] = datetime_array
    ttt['ra'] = ra_array
    ttt['dec'] = dec_array
    ttt['azimuth'] = azimuth_array
    ttt['altitude'] = altitude_array
    ttt['mag'] = mag_array
    ttt['x_image'] = x_image_array
    ttt['y_image'] = y_image_array
    
    return([ttt], 1);


# CAMS functions

In [ ]:
# cams_camera_txt( _camera_file_path )
# #rms_to_dict()
# #camsdict_to_astropy_table(meteor_info: dict, cam_info: dict):
# #cams_dict_list_to_astropy_tables(rms_meteor_dict_list, cams_camera_info):
# cams_to_std(cams_meteor_text, cams_cameras_dict):


def cams_camera_txt(_camera_file_path):
     
    #extract json data
    _cal_txt = open(_camera_file_path).read()
    _cal_lines = _cal_txt.split("\n")
    
    cam_dict = {}
    
    #test if a line is "something = value"
    equals_test = "(.*)=(.*)"
    
    cal_date, cal_time = False, False
    
    for cal_line in _cal_lines :
        if not regex.match(equals_test, cal_line):
            continue
                
        temp_regex_results = regex.search(equals_test, cal_line)
        cal_term       = temp_regex_results[1].strip()
        cal_term_value = temp_regex_results[2].strip()
        L = len(cal_term_value)
        
        print(cal_term, " \t:", cal_term_value)    
        
        if regex.match("\d+", cal_term_value) and len(regex.search("\d+", cal_term_value)[0]) == L:
            cam_dict.update({ cal_term: int(cal_term_value) })
            print("Integer detected")
            continue
        
        if regex.match("[0-9.+-]+", cal_term_value) and len(regex.search("[0-9.+-]+", cal_term_value)[0]) == L:
            cam_dict.update({ cal_term: float(cal_term_value) })
            print("Float detected")
            continue
        
        if regex.match("\d{2}/\d{2}/\d{4}", cal_term_value):
            cal_date_str = regex.search("\d{2}/\d{2}/\d{4}", cal_term_value)[0]
            cal_date = datetime.strptime(cal_date_str,"%m/%d/%Y").strftime("%Y-%m-%d")
            print("Date detected")
            continue
        
        if regex.match("\d{2}:\d{2}:\d{2}.\d{3}", cal_term_value):
            cal_time = regex.search("\d{2}:\d{2}:\d{2}.\d{3}", cal_term_value)[0]
            print("Time detected")
            continue
            
        if cal_term == "FOV dimension hxw (deg)":
            cal_fov_hw = regex.search("([+\-0-9.]+)\s*x\s*([+\-0-9.]+)", cal_term_value)
            print(cal_fov_hw)
            cam_dict.update({
                "FOV dimension hxw (deg)": cal_fov_hw[0],
                'FOV height (deg)' : float(cal_fov_hw[1]),
                'FOV width (deg)'  : float(cal_fov_hw[2]),
            })
            continue
            
        cam_dict.update({cal_term: cal_term_value})
                        
    if cal_date and cal_time:
        cal_timestamp_string = cal_date + "T" + cal_time + "000"
        cal_timestamp = datetime.strptime(cal_timestamp_string, ISO_FORMAT)
        cam_dict.update({"timestamp": cal_timestamp})
            
    print("Got camera data ")
    
    return cam_dict
        
def camsdict_to_astropy_table(meteor_info: dict, cam_info: dict):

    # cam:
    # 
    """
    {
     'Camera number': 3814,
     'Longitude +west (deg)': -5.39928,
     'Latitude +north (deg)': 49.81511,
     'Height above WGS84 (km)': 0.4444,
     'FOV height (deg)'
     'FOV width (deg)'
     'FOV dimension hxw (deg)': '46.93 x   88.25',
     'Plate scale (arcmin/pix)': 3.948,
     'Plate roll wrt Std (deg)': 350.213,
     'Cam tilt wrt Horiz (deg)': 2.656,
     'Frame rate (Hz)': 25.0,
     'Cal center RA (deg)': 50.593,
     'Cal center Dec (deg)': 74.131,
     'Cal center Azim (deg)': 347.643,
     'Cal center Elev (deg)': 36.964,
     'Cal center col (colcen)': 640.0,
     'Cal center row (rowcen)': 360.0,
     'Cal fit order': 201,
     'Camera description': 'None',
     'Lens description': 'None',
     'Focal length (mm)': 0.0,
     'Focal ratio': 0.0,
     'Pixel pitch H (um)': 0.0,
     'Pixel pitch V (um)': 0.0,
     'Spectral response B': 0.45,
     'Spectral response V': 0.7,
     'Spectral response R': 0.72,
     'Spectral response I': 0.5,
     'Vignetting coef(deg/pix)': 0.0,
     'Gamma': 1.0,
     'Xstd, Ystd': 'Radialxy2Standard( col, row, colcen, rowcen, Xcoef, Ycoef )',
     'x': 'col - colcen',
     'y': 'rowcen - row',
     'Mean O-C': '0.000 +-   0.000 arcmin',
     'Magnitude': '-2.5 ( C + D (logI-logVig) )   fit logFlux vs. Gamma (logI-logVig), mV <  6.60',
     'A': 10.0,
     'B': -2.5,
     'C': -4.0,
     'D': 1.0,
     'logVig': 'log( cos( Vignetting_coef * Rpixels * pi/180 )^4 )',
     'timestamp': datetime.datetime(2019, 5, 14, 20, 56, 33, 531000)
    }
    """
    #
    # meteor:
    # [{
    # cam, meteor, segments, fps, hnr, mle, bin, pix/fm, rho, phi, 
    # file_name, file_prefix, timestamp, duration, min_magnitude, max_intensity, 
    # frames: [{
    #     frame, timestamp, col, row, ra, dec, azim, elev, inten, mag
    #   }]
    # }]
    ##
    
    # Now get metadata
    
    #location
    obs_longitude = float(cam_info['Longitude +west (deg)'])
    obs_latitude  = float(cam_info['Latitude +north (deg)'])
    obs_elevation = 1000 * float(cam_info['Height above WGS84 (km)']) # to metres
    
    
    #camera station site name
    location   = str(cam_info['Camera number'])
    telescope  = str(cam_info['Camera number']).zfill(6) #no spaces or special characters

    #observer and instrument
    origin    = "CAMS"  # or other formal network names
    observer  = str(cam_info['Camera number']).zfill(6)
    instrument = cam_info['Camera description']
    lens = cam_info['Lens description']
    image_file = meteor_info['file_name']
    astrometry_number_stars = 0
    cx = 2 * int(cam_info['Cal center col (colcen)'])
    cy = 2 * int(cam_info['Cal center row (rowcen)'])
     
    # calculate event timings - file timestamp
    timestamp = cam_info['timestamp'].strftime(ISO_FORMAT)[:-3]
    
    # frame rate and beginning of clip
    frame_rate = float(meteor_info['fps'])
    meteor_duration = meteor_info['meteor_duration']
    isodate_start_time = meteor_info['frames'][ 0]['timestamp']
    isodate_end_time   = meteor_info['frames'][-1]['timestamp']
    isodate_midpoint_time = isodate_start_time + (isodate_end_time - isodate_start_time)/2
    isodate_start = isodate_start_time.strftime(ISO_FORMAT)   
    isodate_end = isodate_end_time.strftime(ISO_FORMAT)   
    isodate_midpoint = isodate_midpoint_time.strftime(ISO_FORMAT)   

    
    
   # construction of the metadata dictionary
    meta_dic = {
        'obs_latitude': obs_latitude,
        'obs_longitude': obs_longitude,
        'obs_elevation': obs_elevation,
        'origin': origin,
        'location': location,
        'telescope': telescope,
        'camera_id': telescope,
        'observer': observer,
        'comment': '',            
        'instrument': instrument,
        'lens': lens,
        'cx' : cx,     
        'cy' : cy,     
        'photometric_band' : 'Unknown',     
        'image_file' : image_file,
        'isodate_start_obs': isodate_start,
        'isodate_calib' : isodate_midpoint,
        'exposure_time': meteor_duration,
        'astrometry_number_stars' : astrometry_number_stars,
        #'photometric_zero_point': 0.0,
        #'photometric_zero_point_uncertainty': 0.0,
        'mag_label': 'mag',
        'no_frags': 1,
        'obs_az': float(cam_info['Cal center Azim (deg)']),     
        'obs_ev': float(cam_info['Cal center Elev (deg)']),     
        'obs_rot': float(cam_info['Cam tilt wrt Horiz (deg)']),     
        'fov_horiz': float(cam_info['FOV width (deg)']),     
        'fov_vert': float(cam_info['FOV height (deg)']),     
    }
    
    # initialise table
    ttt = Table()        
    #Update the table metadata
    ttt.meta.update(meta_dic)   
   

    #create time and main data arrays
    # Datetime is ISO 8601 UTC format
    datetime_array = []
    # Azimuth are East of North, in degrees
    azimuth_array  = []
    # Altitudes are geometric (not apparent) angles above the horizon, in degrees
    altitude_array = []
 
    #right ascension and declination coordinates
    ra_array  = []
    dec_array = []    
    x_array = []     
    y_array = []      
    mag_array = []    

    
    nlines = len(meteor_info["frames"])
    print('nlines= ',nlines)
    
    for i in range(nlines):
        obs = meteor_info["frames"][i]
        
        azimuth_array.append(  float(obs['azim']) )
        altitude_array.append( float(obs['elev']) )
        datetime_array.append( obs['timestamp'].strftime(ISO_FORMAT)   )
        ra_array.append( float(obs['ra']) )
        dec_array.append( float(obs['dec']) )
        x_array.append( float(obs['col']))     
        y_array.append( float(obs['row']))     
        mag_array.append( float(obs['mag']))   
    
    ## Populate the table with the data created to date
    # create columns
    ttt['datetime'] = datetime_array
    ttt['ra']   = ra_array  * u.degree
    ttt['dec']  = dec_array * u.degree    
    ttt['azimuth']  = azimuth_array  * u.degree
    ttt['altitude'] = altitude_array * u.degree    
    ttt['mag']  =   mag_array     
    ttt['x_image']  = x_array     
    ttt['y_image']  = x_array     
    
    return ttt;

    
def cams_dict_list_to_std(rms_meteor_dict_list, cams_camera_info):
    #get an astropy table list
    ttt_list = []
    for meteor_info in rms_meteor_dict_list:
        # get info for each
        if not meteor_info:
            print("Empty Entry : ", meteor_info, " - Likely due to merging")
            continue
            
        file_prefix = meteor_info['file_prefix']
        
        # convert and add to list
        singular_ttt = camsdict_to_astropy_table(meteor_info, cams_camera_info)
        ttt_list.append(singular_ttt)

    return ttt_list
    
    
def cams_to_std(cams_meteor_text, cams_cameras_dict):
    
    meteor_dict_list = rms_to_dict(cams_meteor_text);
    
    ttt_list = cams_dict_list_to_std( meteor_dict_list, cams_cameras_dict)
    
    return ttt_list, len(ttt_list);

# MetRec functions

In [ ]:
"""
# List of info in .log in _metrec_cfg

AutoConfiguration 	 -  yes
FrameGrabberType 	 -  Meteor II
FrameGrabberDeviceNumber 	 -  1
VideoSignalType 	 -  PAL
InterlacedVideo 	 -  no
TimeBase 	 -  current time
TimeDriftCorrection 	 -  0.0 s/h
TimeZoneCorrection 	 -  0 h
DSTCorrection 	 -  no
DateBase 	 -  current date
DateCorrection 	 -  yes
RecognitionEndTime 	 -  6 h 30 m 0 s
AutoRestart 	 -  no
QuitBehaviour 	 -  quit without confirmation
WaitForDusk 	 -  no
MaximumSolarAltitude 	 -  -16 $
MinimumLunarDistance 	 -  0 $
PosDriftCorrection 	 -  X/Y
PosDriftHistory 	 -  metrec.pos
FrameBufferCount 	 -  300 frame(s)
DelayTime 	 -  0 ms
DisplayRefreshRate 	 -  2
InternalResolution 	 -  max
MeteorElongation 	 -  1
StartThreshold 	 -  1.50
ConstantThreshold 	 -  no
RecognitionThreshold 	 -  0.85
FloorThreshold 	 -  0.50
ThresholdHistory 	 -  metrec.thr
FlashThreshold 	 -  20
FlashRecoveryFrameCount 	 -  50 frame(s)
SaveFlashImage 	 -  no
SaveBackgroundRate 	 -  never
MinimumFrameCount 	 -  3 frame(s)
Beep 	 -  no
SendSerialPing 	 -  yes
SerialPingPort 	 -  1
SerialPingType 	 -  ABEI
MinimumMeteorVelocity 	 -  1.0 $/s
MaximumMeteorVelocity 	 -  50.0 $/s
PositionAngleOffset 	 -  0 $
UseInputMask     -  yes
InputMask        -  c:\cilbo\metrec\config\ICC7mask.bmp
DarkField        -  dark.bmp
UseOldFlatField  -  no
NewFlatField 	 -  metrec.ffd
FlatFieldSmooth 	 -  2
FlatFieldSmoothDir 	 -  symmetric
SensitivityImage 	 -  metrec.bmp
TracingImage 	 -  ????????.bmp
TimeStamp 	 -  date and time
TimeStampXPosition 	 -  384
TimeStampYPosition 	 -  288
SaveSingleFrames 	 -  bright only
SingleFrameBrightness 	 -  0.0
SingleFrameDuration 	 -  0.5
SaveMeteorBand 	 -  yes
SaveSumImage 	 -  yes
SaveMeteorData 	 -  yes
SavePreFrameCount 	 -  3 frame(s)
SavePostFrameCount 	 -  3 frame(s)
SavePostFrameBright 	 -  30 frame(s)
RealTimeFluxUpload 	 -  no
CameraName 	 -  ICC7
BaseDirectory 	 -  c:\cilbo\data\ICC7\
FileNameRule 	 -  hhmmssff.bmp
ClockSync 	 -  no
EquatorialCoordinates 	 -  yes
ReferenceStars 	 -  20190903.ref
MaximumMeteorTilt 	 -  0 $
MaximumMeteorShift 	 -  0 $
CreatePosDatEntry 	 -  yes
Operation mode 	 -  unguided
Reference date 	 -  2019/09/03
Reference time 	 -  23:00:00
Site code 	 -  15556
Longitude 	 -  -16.509171 $
Latitude 	 -  28.298901 $
Altitude 	 -  2400 m
Noise Level 	 -  5.0
Maximum Star Diameter 	 -  4.0
Minimum Star Diameter 	 -  1.0
Video brightness 	 -  128
Video contrast 	 -  128
Gamma correction 	 -  1.00
Order of plate constants 	 -  3
Center of plate RA 	 -  18.0080 h
Center of plate DE 	 -  34.5535 $
Center of plate Alt 	 -  54.6 $
Center of plate Az 	 -  290.8 $
Size of field of view 	 -  30.5 x 23.0 $
O-C RefStar1 	 -  msqe= 0.55'  l1o= 0.63'  -0.00 mag  (B-V= 1.40 mag)
...
O-C RefStar51 	 -  msqe= 1.81'  l1o= 2.08'   0.10 mag  (B-V= 1.10 mag)
Mean Squared O-C 	 -  msqe= 1.66'  l1o= 2.04'   0.41 mag
Photometric equation 	 -  -2.326 log(pixelsum) + 8.390
Color index correction 	 -  -0.127 (B-V) + 0.063
Nominal lim. magnitude 	 -  5.7 mag
Total collection area 	 -  682 deg^2 / 4196 km^2 @ 100 km alt
Corrected total collection area 	 -  2377 km^2
Number of active meteor showers (2019/11/01) 	 -  3
"""

"""
# List of info in inf

'#', 'time', 'bright', 'x', 'y', 'alpha', 'delta', 'c_x', 'c_y', 'c_alpha', 'c_delta', 'use', 'timestamp'
"""


def metrec_to_standard(inf, log):

    cfg = log._metrec_cfg
    
    def getFloat(numstr):
        return float( regex.match('[+\-0-9.]+',numstr)[0] )      
    
    #location
    obs_longitude = getFloat(cfg['Longitude'])
    obs_latitude  = getFloat(cfg['Latitude'])
    obs_elevation = getFloat(cfg['Altitude'])

    #camera station site name
    location   = str(cfg['Site code'])
    telescope  = str(cfg['CameraName']).zfill(6) #no spaces or special characters

    #observer and instrument
    origin     = "MetRec"  # or other formal network names
    observer   = cfg['CameraName']
    instrument = cfg['CameraName']
    lens       = 'unknown'
    image_file = inf.path
    astrometry_number_stars = 0
    
    if cfg['TimeStamp'] == 'none':
        cx = 0
        cy = 0
    else:    
        cx = int(cfg['TimeStampXPosition'])
        cy = int(cfg['TimeStampYPosition'])
    
    
    # calculate event timings - file timestamp
    timestamp = inf['timestamp'][0]
    
    meteor_duration    = inf['timestamp'][0]
    isodate_start = inf['timestamp'][0]
    isodate_end   = inf['timestamp'][-1]
    start_datetime = datetime.strptime( isodate_start, ISO_FORMAT )
    end_datetime   = datetime.strptime( isodate_end,   ISO_FORMAT )
    meteor_duration    = (end_datetime - start_datetime)

    isodate_midpoint_time = start_datetime + meteor_duration/2
    isodate_midpoint = isodate_midpoint_time.strftime(ISO_FORMAT)   
    
    meteor_duration = meteor_duration.total_seconds()

    # get FOV x and y
    cfg_fov = regex.search("([+\-0-9.]+)\s*x\s*([+\-0-9.]+)", cfg['Size of field of view'])
    
   # construction of the metadata dictionary
    meta_dic = {
        'obs_latitude': obs_latitude,
        'obs_longitude': obs_longitude,
        'obs_elevation': obs_elevation,
        'origin': origin,
        'location': location,
        'telescope': telescope,
        'camera_id': telescope,
        'observer': observer,
        'comment': '',            
        'instrument': instrument,
        'lens': lens,
        'cx' : cx,     
        'cy' : cy,     
        'photometric_band' : 'Unknown',     
        'image_file' : image_file,
        'isodate_start_obs': isodate_start,
        'isodate_calib' : isodate_midpoint,
        'exposure_time': meteor_duration,
        'astrometry_number_stars' : astrometry_number_stars,
        #'photometric_zero_point': 0.0,
        #'photometric_zero_point_uncertainty': 0.0,
        'mag_label': 'mag',
        'no_frags': 1,
        'obs_az': getFloat(cfg['Center of plate Az']),     
        'obs_ev': getFloat(cfg['Center of plate Alt']),     
        'obs_rot': 0.0,   #float(cam_info['Cam tilt wrt Horiz (deg)']), # not in MetRec?   
        'fov_horiz': float(cfg_fov[1]),     
        'fov_vert': float(cfg_fov[2]),     
    }
    
    # initialise table
    ttt = Table()        
    #Update the table metadata
    ttt.meta.update(meta_dic) 
    
    
    # Meteor Info
    # Datetime is ISO 8601 UTC format
    metrec_index_array = []
    datetime_array = []
    azimuth_array  = [] # Azimuth are East of North, in degrees
    altitude_array = [] # Altitudes are geometric (not apparent) angles above the horizon, in degrees
 
    #right ascension and declination coordinates
    ra_array  = []
    dec_array = []    

    x_array = []     
    y_array = []      
    mag_array = []  
    
    # start of J2000 epoch
    ts = datetime.strptime("2000-01-01T12:00:00.000",ISO_FORMAT)
    start_epoch = datetime2JD(ts)


    for i in range(len(inf['use'])):
        if inf['use'][i] == True:
            # time and location
            metrec_index_array.append( i )
            
            temp_timestamp = inf['timestamp'][i]
            temp_datetime  = datetime.strptime(temp_timestamp, ISO_FORMAT)

            # RA is in hours, so multiply by 15
            ra = 15 * float(inf['alpha'][i])
            dec= float(inf['delta'][i])
            
            # RA and DEC are in J2000 epoch.  Precess to epoch of date, then convert to Az and Alt using RMS code
            JD = datetime2JD(temp_datetime)
            temp_ra, temp_dec = equatorialCoordPrecession(start_epoch, JD, ra, dec)  
            temp_azim, temp_elev = raDec2AltAz(temp_ra, temp_dec, JD, obs_latitude, obs_longitude)

            #right ascension and declination coordinates read, alt and az need to be calculated
            temp_azim, temp_alt = raDec2AltAz(temp_ra, temp_dec, JD, obs_latitude, obs_longitude)
            
            datetime_array.append( temp_timestamp )
            ra_array.append( ra  )
            dec_array.append( dec )
            azimuth_array.append( temp_azim )
            altitude_array.append( temp_alt )  

            x_array.append( float(inf['x'][i]) )
            y_array.append( float(inf['y'][i]) )
            
            # astronomical magnitude of the brightness 
            if inf['bright'][i] == None :
                mag_array.append( 99.9 ) 
            else:    
                mag_array.append( float(inf['bright'][i]) ) 

    ## Populate the table with the data created to date
    # create columns
    ttt['datetime'] = datetime_array
    ttt['ra']   = ra_array  * u.degree
    ttt['dec']  = dec_array * u.degree    
    ttt['azimuth']  = azimuth_array  * u.degree
    ttt['altitude'] = altitude_array * u.degree    
    ttt['mag']  =   mag_array
    ttt['x_image']  = x_array
    ttt['y_image']  = x_array

    
    return [ttt], 1;

# All Sky Cams functions

In [ ]:
def get_as7_stations(station_str):
    # get a table of AllSky7 camera locations
    # return the table, plus the index corresponding to "station_str"

    stations_file_name = 'https://raw.githubusercontent.com/SCAMP99/scamp/master/ALLSKY7_location_list.csv'    

    import requests
    try:
        r = requests.get(stations_file_name)
        loc_table = ascii.read(r.text, delimiter=',')
        print('Filling location table from online index')
    except:
        # create columns for the UK and ROI stations only. 
        # Station,City,Longitude,Latitude,Altitude,Firstlight,Operator
        # AMS101,Birmingham Astronomical Society,-1.846419,52.408080,127,February 2021,Ben Stanley
        # AMS100,Nuneaton,-1.45472222,52.52638889,80,December 2020,Ben Stanley
        # AMS113,Galway,-9.089128,53.274739,31,January 2021,Charlie McCormack
        
        loc_table = Table()
        loc_table['Station'] = 'AMS101','AMS100','AMS113'
        loc_table['City'] = 'Birmingham Astronomical Society','Nuneaton','Galway'
        loc_table['Longitude'] = '-1.846419','-1.45472222','-9.089128'
        loc_table['Latitude'] = '52.40808','52.52638889','53.274739'
        loc_table['Altitude'] = '127','80','31'
        loc_table['Firstlight'] = 'Feb 2021','Dec 2020','Jan 2021'
        loc_table['Operator'] = 'Ben Stanley','Ben Stanley','Charlie McCormack'
        print('Filling location table from known locations')

    no_stations = len(loc_table['Latitude'])

    #The first key may have extra characters in it - if so, rename it.
    for key_name in loc_table.keys(): 
        if 'Station' in key_name:
            if not key_name == 'Station':
                loc_table.rename_column(key_name,'Station')
    
    #print(loc_table)

    i = -1
    for j in range(no_stations):
        if loc_table['Station'][j] == station_str:
            i = j
            break
    
    if i < 0:    
        print('AllSky7 Station name "' + station_str + '" not found.') 
        return([], 0, i);
    

    print('AllSky7 Station name "' + station_str + '" is in row ',i) 
    return(loc_table, no_stations, i);



def allskycams_to_std(json_data, lname):
    # This reads a string which is in AllSkyCams json format and converts it to standard format

    # Set up arrays for point observation data
    datetime_array = []
    datestr_array = []
    azimuth_array = []
    altitude_array = []
    ra_array = []
    dec_array = []
    mag_array = []
    x_image_array = []
    y_image_array = []
    

    # Standard AllSky7 spec
    instrument = 'NST-IPC16C91 - Low Lux SONY STARVIS Sensor Wireless IP Board Camera'
    lens = '4 mm f/1.0'
    cx = 1920
    cy = 1080

    
    # Check the json data to see which format it is in, and extract information accordingly.  
    
    if 'station_name' in json_data :
        # The February 2021 "reduced.json" format
        # if lname.endswith("reduced.json"):
        
        # for key_name in json_data.keys(): 
        #    print(key_name)
        #
        # api_key
        # station_name
        # device_name
        # sd_video_file
        # sd_stack
        # hd_video_file
        # hd_stack
        # event_start_time
        # event_duration
        # peak_magnitude
        # start_az
        # start_el
        # end_az
        # end_el
        # start_ra
        # start_dec
        # end_ra
        # end_dec
        # meteor_frame_data
        # crop_box
        # cal_params            
        # 
        

        no_lines = len(json_data['meteor_frame_data'])
        #print('no_lines = ',no_lines)
        #for k in range(no_lines):
        #    print('\n',json_data['meteor_frame_data'][k])
        #    if k == 3 :
        #        for m in range(len(json_data['meteor_frame_data'][k])):
        #            print("json_data['meteor_frame_data'][",k,"][",m,"] = ",json_data['meteor_frame_data'][k][m])

        for i in range(no_lines):
            f_data = json_data['meteor_frame_data'][i]
            #  "meteor_frame_data": [
            #  [
            #      [0] "2021-02-04 05:42:07.800",
            #      [1] 46, fn
            #      [2] 381, x
            #      [3] 118, y
            #      [4] 10, w
            #      [5] 10, h
            #      [6] 1275, [Number of pixels?]
            #      [7] 348.38983647729816, RA
            #      [8] 65.16531356859444, Dec
            #      [9] 22.88795625855195, az
            #      [10] 33.84381610533057, el
            #   ],

            date_str = f_data[0].replace(' ','T')
            date_time = datetime.strptime(date_str,ISO_FORMAT)
            #print('i=',i,' date_str =',date_str, ' date_time =',date_time)
            datetime_array.append(date_time)
            datestr_array.append(date_str)
            azimuth_array.append(float(f_data[9]))
            altitude_array.append(float(f_data[10]))
            ra_array.append(float(f_data[7]))
            dec_array.append(float(f_data[8]))
            mag_array.append(0.0)
            x_image_array.append(float(f_data[2]))
            y_image_array.append(float(f_data[3]))
        
        meteor_duration = datetime_array[-1] - datetime_array[0]
        print('meteor_duration = ', meteor_duration)
        meteor_duration_float = float(meteor_duration.total_seconds())
        frame_rate = (json_data['meteor_frame_data'][-1][1]- json_data['meteor_frame_data'][0][1]) / meteor_duration_float
        print('frame_rate = ', frame_rate)
    
        time_step = (1 - json_data['meteor_frame_data'][0][1]) / frame_rate 
        isodate_start_time = datetime_array[0] + timedelta(seconds=time_step)
        print('isodate_start_time = ', isodate_start_time)
        isodate_end_time   = datetime_array[-1]
        print('datetime_array[0] = ', datetime_array[0])
        print('isodate_end_time = ', isodate_end_time)

        isodate_midpoint_time = isodate_start_time + (isodate_end_time - isodate_start_time)/2
        print('isodate_midpoint_time = ', isodate_midpoint_time)
        isodate_start = isodate_start_time.strftime(ISO_FORMAT)   
        isodate_end = isodate_end_time.strftime(ISO_FORMAT)   
        isodate_midpoint = isodate_midpoint_time.strftime(ISO_FORMAT)   
        
        
        print('\n getting the list of AS7 stations, call 1')
        station_str = json_data['station_name']
        loc_table, no_stations, row_no = get_as7_stations(station_str)
        #Station,City,Longitude,Latitude,Altitude,Firstlight,Operator
        
        if row_no < 0:   
            # Station data is unavailable
            # Put placeholders for station data
            obs_latitude = -999.9  
            obs_longitude = -999.9  
            obs_elevation = -999.9  
            location = 'Unknown'
            telescope = 'Unknown'
            camera_id = 'Unknown'
            observer = 'Unknown'
        else:
            # Use the information from the lookup table
            device_data = loc_table[row_no]
            
            obs_latitude = float(device_data['Latitude'])  
            obs_longitude = float(device_data['Longitude'])  
            obs_elevation = float(device_data['Altitude'])  
            location = str(device_data['City'])
            telescope = str(station_str)
            camera_id = str(station_str)
            observer = str(device_data['Operator'])
        
        device_data_internal = json_data['cal_params']
        # "cal_params": {
        #     "center_az": 291.03838531805667,
        #     "center_el": 24.91924498460342,
        #     "position_angle": 41.09621614877751,
        #     "pixscale": 155.58669548833825,
        #     "ra_center": "22.498291666666667",
        #     "dec_center": "32.03936111111111",
        #     "user_stars": [

        rstars = len(device_data_internal['user_stars'])
        
        meta_dic = {'obs_latitude': obs_latitude,  
           'obs_longitude': obs_longitude,  
           'obs_elevation': obs_elevation,  
           'origin': 'All Sky Systems',
           'location': location,
           'telescope': telescope,
           'camera_id': camera_id,
           'observer': observer,
           'comment': '',            
           'instrument': instrument,
           'lens': lens,
           'cx' : cx,     
           'cy' : cy,     
           'photometric_band' : 'Unknown',     
           'image_file' : json_data['hd_video_file'],
           'isodate_start_obs': str(isodate_start),  
           'isodate_calib' : str(isodate_midpoint), 
           'exposure_time': meteor_duration_float,    
           'astrometry_number_stars' : rstars,
           #'photometric_zero_point': 0.0,    
           #'photometric_zero_point_uncertainty': 0.0,
           'mag_label': 'no_mag_data',     
           'no_frags': 1,
           'obs_az': float(device_data_internal['center_az']),     
           'obs_ev': float(device_data_internal['center_el']),     
           'obs_rot': 0.0,     
           'fov_horiz': 0.0,     
           'fov_vert': 0.0,     
           }


    elif 'best_meteor' in json_data :
        # is the February 2021 format without station data
        # includes the hacked form with manual dates, az, el 
        print("\n the key 'best_meteor' is in the json data")
            
        #for key_name in json_data.keys(): 
        #    print(key_name)
        # sd_video_file
        # sd_stack
        # sd_objects
        # hd_trim
        # hd_stack
        # hd_video_file
        # hd_objects
        # meteor
        # cp
        # best_meteor            
        #         
        # for key_name in json_data['best_meteor'].keys(): 
        #     print(key_name)
        # 
        #     obj_id
        #     ofns
        #     oxs
        #     oys
        #     ows
        #     ohs
        #     oint
        #     fs_dist
        #     segs
        #     report
        #     ccxs
        #     ccys
        #     dt
        #     ras
        #     decs
        #     azs
        #     els            

        no_lines = len(json_data['best_meteor']['dt'])
        print('no_lines = ',no_lines)
        file_hacked = ('ras' not in json_data['best_meteor'] )
        print('file_hacked = ',file_hacked)

        for i in range(no_lines):
            date_str = (str(json_data['best_meteor']['dt'][i])).replace(' ','T')
            date_time = datetime.strptime(date_str,ISO_FORMAT)
            print('i=',i,' date_str=',date_str, ' date_time=',date_time)
            datetime_array.append(date_time)
            datestr_array.append(date_str)
            azimuth_array.append(float(json_data['best_meteor']['azs'][i]))
            altitude_array.append(float(json_data['best_meteor']['els'][i]))
            if file_hacked :
                x_image_array.append(0.0)
                y_image_array.append(0.0)
                # Do RA and DEC later
            else:    
                x_image_array.append(float(json_data['best_meteor']['ccxs'][i]))
                y_image_array.append(float(json_data['best_meteor']['ccys'][i]))
                ra_array.append(float(json_data['best_meteor']['ras'][i]))
                dec_array.append(float(json_data['best_meteor']['decs'][i]))
            mag_array.append(0.0)
        
        meteor_duration = datetime_array[-1] - datetime_array[0]
        print('meteor_duration = ', meteor_duration)
        meteor_duration_float = float(meteor_duration.total_seconds())
        frame_rate = (json_data['best_meteor']['ofns'][-1]- json_data['best_meteor']['ofns'][0]) / meteor_duration_float
        print('frame_rate = ', frame_rate)
    
        time_step = (1 - json_data['best_meteor']['ofns'][0]) / frame_rate 
        isodate_start_time = datetime_array[0] + timedelta(seconds=time_step)
        print('isodate_start_time = ', isodate_start_time)
        isodate_end_time   = datetime_array[-1]
        print('datetime_array[0] = ', datetime_array[0])
        print('isodate_end_time = ', isodate_end_time)

        isodate_midpoint_time = isodate_start_time + (isodate_end_time - isodate_start_time)/2
        print('isodate_midpoint_time = ', isodate_midpoint_time)
        isodate_start = isodate_start_time.strftime(ISO_FORMAT)   
        isodate_end = isodate_end_time.strftime(ISO_FORMAT)   
        isodate_midpoint = isodate_midpoint_time.strftime(ISO_FORMAT)   
        

        # Now work out which station it is.  
        # there is very little station info in the data file
        station_str = ''
        if 'archive_file' in json_data :
            # The archive filename contains the station name
            arch_str = str(json_data['archive_file'])
            arch_list = arch_str.split('/')
            for i in range (len(arch_list)):
                if ('AMS' in arch_list[i]) and (not arch_list[i] == 'AMS2'):
                    station_str = arch_list[i]
                    break

        if len(station_str) < 1:            
            station_str = input("\nWhich AllSky7 station is this data from (e.g. AMS100) :")

        if len(station_str) < 1:   
            row_no = -1
        else:    
            print('\n getting the list of AS7 stations, call 2')
            loc_table, no_stations, row_no = get_as7_stations(station_str)
            #Station,City,Longitude,Latitude,Altitude,Firstlight,Operator
            
        if row_no < 0:   
            # Station data is unavailable
            # Put placeholders for station data
            obs_latitude = -999.9  
            obs_longitude = -999.9  
            obs_elevation = -999.9  
            location = 'Unknown'
            telescope = 'Unknown'
            camera_id = 'Unknown'
            observer = 'Unknown'
        else:
            # Use the information from the lookup table
            device_data = loc_table[row_no]
            
            obs_latitude = float(device_data['Latitude'])  
            obs_longitude = float(device_data['Longitude'])  
            obs_elevation = float(device_data['Altitude'])  
            location = str(device_data['City'])
            telescope = str(station_str)
            camera_id = str(station_str)
            observer = str(device_data['Operator'])
            
            
        device_data_internal = json_data['cp']
        # "cp": {
        #     "center_az": 345.6233888888889,
        #     "center_el": 19.169500000000003,
        #     "position_angle": 13.914659642573255,
        #     "pixscale": 155.901484181,
        #     "ra_center": "310.7067083333333",
        #     "dec_center": "54.69519444444444",
        #     "user_stars": [
        #     [

        
        if file_hacked :
            rstars = 0
            comment = 'Reconstructed from basic az and alt data. No XY data'
            # Now add RA and DEC
        
            # start of J2000 epoch
            ts = datetime.strptime("2000-01-01T12:00:00.000",ISO_FORMAT)
            start_epoch = datetime2JD(ts)

            for i in range(no_lines):
                az   = float(azimuth_array[i])
                elev = float(altitude_array[i])
        
                time_stamp = datestr_array[i]
                ts = datetime.strptime(time_stamp,ISO_FORMAT)
                JD = datetime2JD(ts)
        
                # USE Az and Alt to calculate correct RA and DEC in epoch of date, then precess back to J2000
                temp_ra, temp_dec = altAz2RADec(az, elev, JD, obs_latitude, obs_longitude)
                temp_ra, temp_dec = equatorialCoordPrecession(JD, start_epoch, temp_ra, temp_dec) 
                ra_array.append(temp_ra )
                dec_array.append(temp_dec )        
        else:    
            rstars = len(device_data_internal['user_stars'])
            comment = ''

        
        meta_dic = {'obs_latitude': obs_latitude,  
           'obs_longitude': obs_longitude,  
           'obs_elevation': obs_elevation,  
           'origin': 'All Sky Systems',
           'location': location,
           'telescope': telescope,
           'camera_id': camera_id,
           'observer': observer,
           'comment': comment,            
           'instrument': instrument,
           'lens': lens,
           'cx' : cx,     
           'cy' : cy,     
           'photometric_band' : 'Unknown',     
           'image_file' : json_data['hd_video_file'],
           'isodate_start_obs': str(isodate_start),  
           'isodate_calib' : str(isodate_midpoint), 
           'exposure_time': meteor_duration_float,    
           'astrometry_number_stars' : rstars,
           #'photometric_zero_point': 0.0,    
           #'photometric_zero_point_uncertainty': 0.0,
           'mag_label': 'no_mag_data',     
           'no_frags': 1,
           'obs_az': float(device_data_internal['center_az']),     
           'obs_ev': float(device_data_internal['center_el']),     
           'obs_rot': 0.0,     
           'fov_horiz': 0.0,     
           'fov_vert': 0.0,     
           }
            
        
    elif 'info' in json_data :
        # is the July 2020 format
        print('\n July 2020 format')
        for key_name in json_data.keys(): 
            print(key_name)
        # info
        # frames
        # report
        # sync
        # calib 

        camera_id = json_data['info']['station']
        location = str(json_data['info']['station'])
        telescope = json_data['info']['device']
        cx = int(json_data['calib']['img_dim'][0])     
        cy = int(json_data['calib']['img_dim'][1])     
        rstars = len(json_data['calib']['stars']) 
        no_lines = len(json_data['frames'])

        # Work out who the observer was, if possible
        loc_table, no_stations, row_no = get_as7_stations(camera_id)
        #Station,City,Longitude,Latitude,Altitude,Firstlight,Operator
        if row_no >= 0:
            observer = str(loc_table[row_no]['Operator'])
        else:
            observer = location + ' ' + telescope

        #print("\n len(json_data['frames']) = ",no_lines)

        for i in range(no_lines):
            f_data = json_data['frames'][i]
            #print("\n json_data['frames'][",i,"] = ",f_data)
            #json_data['frames'][ 4 ] =  {'fn': 57, 'x': 727, 'y': 667, 'w': 11, 'h': 11, 'dt': '2020-07-09 01:27:36.400', 
            #                             'az': 51.85325570513405, 'el': 31.17297922001948, 'ra': 317.34699971600514, 
            #                             'dec': 47.48858399651199}

            date_str = f_data['dt'].replace(' ','T')
            date_time = datetime.strptime(date_str,ISO_FORMAT)
            #print('i=',i,' date_str =',date_str, ' date_time =',date_time)
            datetime_array.append(date_time)
            datestr_array.append(date_str)
            azimuth_array.append(float(f_data['az']))
            altitude_array.append(float(f_data['el']))
            ra_array.append(float(f_data['ra']))
            dec_array.append(float(f_data['dec']))
            mag_array.append(0.0)
            x_image_array.append(float(f_data['x']))
            y_image_array.append(float(f_data['y']))

        meteor_duration = datetime_array[-1] - datetime_array[0]
        print('meteor_duration = ', meteor_duration)
        meteor_duration_float = float(meteor_duration.total_seconds())
        frame_rate = (json_data['frames'][-1]['fn']- json_data['frames'][0]['fn']) / meteor_duration_float
        print('frame_rate = ', frame_rate)
    
        time_step = (1 - json_data['frames'][0]['fn']) / frame_rate 
        isodate_start_time = datetime_array[0] + timedelta(seconds=time_step)
        print('isodate_start_time = ', isodate_start_time)
        isodate_end_time   = datetime_array[-1]
        print('datetime_array[0] = ', datetime_array[0])
        print('isodate_end_time = ', isodate_end_time)

        isodate_midpoint_time = isodate_start_time + (isodate_end_time - isodate_start_time)/2
        print('isodate_midpoint_time = ', isodate_midpoint_time)
        isodate_start = isodate_start_time.strftime(ISO_FORMAT)   
        isodate_end = isodate_end_time.strftime(ISO_FORMAT)   
        isodate_midpoint = isodate_midpoint_time.strftime(ISO_FORMAT)   

        # construction of the metadata dictionary
        device_data = json_data['calib']['device']                    
        meta_dic = {'obs_latitude': float(device_data['lat']),  
            'obs_longitude': float(device_data['lng']),  
            'obs_elevation': float(device_data['alt']),  
            'origin': 'All Sky Systems',
            'location': location,
            'telescope': telescope,
            'camera_id': camera_id,
            'observer': observer,
            'comment': '',            
            'instrument': instrument,
            'lens': lens,
            'cx' : cx,     
            'cy' : cy,     
            'photometric_band' : 'Unknown',     
            'image_file' : json_data['info']['org_hd_vid'],
            'isodate_start_obs': str(isodate_start),
            'isodate_calib' : str(isodate_midpoint),
            'exposure_time': meteor_duration_float,
            'astrometry_number_stars' : rstars,
            #'photometric_zero_point': 0.0,    
            #'photometric_zero_point_uncertainty': 0.0,
            'mag_label': 'no_mag_data',     
            'no_frags': 1,
            'obs_az': float(device_data['center']['az']),     
            'obs_ev': float(device_data['center']['el']),     
            'obs_rot': 0.0,     
            'fov_horiz': 0.0,     
            'fov_vert': 0.0,     
            }
    
    else:        
        print('\n Json format not recognised')
        return([], 0);
        
    
    # initialise table
    ttt = Table()        
    #Update the table metadata
    ttt.meta.update(meta_dic)   
    
    ttt['datetime'] = datestr_array
    ttt['ra'] = ra_array
    ttt['dec'] = dec_array
    ttt['azimuth'] = azimuth_array
    ttt['altitude'] = altitude_array
    ttt['no_mag_data'] = mag_array
    ttt['x_image'] = x_image_array
    ttt['y_image'] = y_image_array
    
    return([ttt], 1);



def std_to_allskycams(ttt):

    info = {}
    info['station'] = ttt.meta['location']   
    info['device'] =  ttt.meta['telescope']    
    info['org_hd_vid'] = ttt.meta['image_file']    

    
    # work out the frame rate of the observations in the table.  
    # this code is long-form as it was copied across from the UFO conversion
    start_time = Time(ttt['datetime'][0])  
    start_time_str = str(ttt['datetime'][0])  
    nlines = len(ttt['datetime'])    
    cumu_times = []
    step_sizes = []
    last_sec = 0.0
    for i in range(nlines):
        sec = get_secs(Time(ttt['datetime'][i]),start_time)
        cumu_times.append(sec)
        sec_rounded = sec
        time_change = int(round(1000*(sec_rounded - last_sec),0))
        if i>0 and (time_change not in step_sizes):
            step_sizes.append(time_change)
        last_sec = sec_rounded
    #now test for common framerates
    # likely framerates are 20 (DFN), 25 (UFO) or 30 (FRIPON) fps
    smallest = min(step_sizes)
    if (smallest==33 or smallest == 34 or smallest == 66 or smallest == 67):
        frame_rate = 30.0
    elif (smallest >= 39 and smallest <= 41):
        frame_rate = 25.0
    elif (smallest >= 49 and smallest <= 51):
        frame_rate = 20.0
    else:
        # non-standard framerate
        # gcd is the greatest common divisor of all of the steps, in milliseconds.  
        # Note - if gcd <= 10 it implies frame rate >= 100 fps, which is probably caused by a rounding error
        gcd = array_gcd(step_sizes)
        frame_rate = 1000.0/float(gcd)
    frame_step = 1/frame_rate
    #work out the head, tail and first frame number
    head_sec = round(-get_secs(Time(ttt.meta['isodate_start_obs']),start_time),6)
    head = int(round(head_sec / frame_step,0))
    fs = head + 1
    fN = 1+int(round(sec/frame_step,0))
    fe = fs + fN -1
    sN = nlines
    sec = round(sec, 4)
    
    # work out number of frames-equivalent and tail
    mid_sec = round(head_sec + get_secs(Time(ttt.meta['isodate_calib']),start_time),6)
    clip_sec = round(max(min(2*mid_sec,30.0),(fe-1)*frame_step),6)   
    no_frames = int(round(clip_sec/frame_step,0)) + 1
    tail = max(0,no_frames - (head + fN))
    no_frames = head + fN + tail
    
    frames = []
    for i in range(nlines):
        frame = {}
        frame['fn'] = fs + int(round(cumu_times[i]/frame_step,0))
        frame['x'] = int(round(ttt[i]['x_image'],0))
        frame['y'] = int(round(ttt[i]['y_image'],0))
        frame['w'] = 0
        frame['h'] = 0
        frame['dt'] = ttt[i]['datetime'].replace('T',' ')
        frame['az'] = ttt[i]['azimuth']
        frame['el'] = ttt[i]['altitude']
        frame['ra'] = ttt[i]['ra']
        frame['dec'] = ttt[i]['dec']
        frames.append(frame)

    center_dic = {}
    center_dic['az'] = ttt.meta['obs_az']        
    center_dic['el'] = ttt.meta['obs_ev']        

    device = {}        
    device['center'] = center_dic
    device['alt'] = str(ttt.meta['obs_elevation'])
    device['lat'] = str(ttt.meta['obs_latitude'])
    device['lng'] = str(ttt.meta['obs_longitude'])

    calib = {}
    calib['device'] = device        
    calib['img_dim'] = [ttt.meta['cx'], ttt.meta['cy']]  
            
    
    # assemble a dictionary with the right data structure    
    json_dict = {}
    json_dict['info'] = info    
    json_dict['frames'] = frames    
    json_dict['calib'] = calib    

    # convert the dictionary to a string 
    json_str = json.dumps(json_dict, ensure_ascii=True, indent=4)
    
    return json_str


# RA & DEC <==> Az Alt conversion, from RMS (c) Denis Vida

In [ ]:
""" 
A set of tools of working with meteor data. 
Includes:
    - Julian date conversion
    - LST calculation
    - Coordinate transformations
    - RA and Dec precession correction
    - ...
    
"""

# The MIT License

# Copyright (c) 2016 Denis Vida

# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:

# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.

# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN
# THE SOFTWARE.

import math
from datetime import datetime, timedelta, MINYEAR


### CONSTANTS ###

# Define Julian epoch
JULIAN_EPOCH = datetime(2000, 1, 1, 12) # noon (the epoch name is unrelated)
J2000_JD = timedelta(2451545) # julian epoch in julian dates

class EARTH_CONSTANTS(object):
    """ Holds Earth's shape parameters. """

    def __init__(self):

        # Earth elipsoid parameters in meters (source: IERS 2003)
        self.EQUATORIAL_RADIUS = 6378136.6
        self.POLAR_RADIUS = 6356751.9
        self.RATIO = self.EQUATORIAL_RADIUS/self.POLAR_RADIUS
        self.SQR_DIFF = self.EQUATORIAL_RADIUS**2 - self.POLAR_RADIUS**2

# Initialize Earth shape constants object
EARTH = EARTH_CONSTANTS()


#################


### Time conversions ###


def JD2LST(julian_date, lon):
    """ Convert Julian date to Local Sidreal Time and Greenwich Sidreal Time. 
    
    Arguments;
        julian_date: [float] decimal julian date, epoch J2000.0
        lon: [float] longitude of the observer in degrees
    
    Return:
        [tuple]: (LST, GST): [tuple of floats] a tuple of Local Sidreal Time and Greenwich Sidreal Time 
            (degrees)
    """

    t = (julian_date - J2000_JD.days)/36525.0

    # Greenwich Sidreal Time
    GST = 280.46061837 + 360.98564736629 * (julian_date - 2451545) + 0.000387933 *t**2 - ((t**3) / 38710000)
    GST = (GST+360) % 360

    # Local Sidreal Time
    LST = (GST + lon + 360) % 360
    
    return LST, GST


def date2JD(year, month, day, hour, minute, second, millisecond=0, UT_corr=0.0):
    """ Convert date and time to Julian Date with epoch J2000.0. 
    @param year: [int] year
    @param month: [int] month
    @param day: [int] day of the date
    @param hour: [int] hours
    @param minute: [int] minutes
    @param second: [int] seconds
    @param millisecond: [int] milliseconds (optional)
    @param UT_corr: [float] UT correction in hours (difference from local time to UT)
    @return :[float] julian date, epoch 2000.0
    """

    # Convert all input arguments to integer (except milliseconds)
    year, month, day, hour, minute, second = map(int, (year, month, day, hour, minute, second))

    # Create datetime object of current time
    dt = datetime(year, month, day, hour, minute, second, int(millisecond*1000))

    # Calculate Julian date
    julian = dt - JULIAN_EPOCH + J2000_JD - timedelta(hours=UT_corr)
    
    # Convert seconds to day fractions
    return julian.days + (julian.seconds + julian.microseconds/1000000.0)/86400.0



def datetime2JD(dt, UT_corr=0.0):
    """ Converts a datetime object to Julian date. 
    Arguments:
        dt: [datetime object]
    Keyword arguments:
        UT_corr: [float] UT correction in hours (difference from local time to UT)
    Return:
        jd: [float] Julian date
    """

    return date2JD(dt.year, dt.month, dt.day, dt.hour, dt.minute, dt.second, dt.microsecond/1000.0, 
        UT_corr=UT_corr)






############################


### Spatial coordinates transformations ###

def altAz2RADec(azim, elev, jd, lat, lon):
    """ Convert azimuth and altitude in a given time and position on Earth to right ascension and 
        declination. 
    Arguments:
        azim: [float] azimuth (+east of due north) in degrees
        elev: [float] elevation above horizon in degrees
        jd: [float] Julian date
        lat: [float] latitude of the observer in degrees
        lon: [float] longitde of the observer in degrees
    Return:
        (RA, dec): [tuple]
            RA: [float] right ascension (degrees)
            dec: [float] declination (degrees)
    """

    azim = np.radians(azim)
    elev = np.radians(elev)
    lat = np.radians(lat)
    lon = np.radians(lon)
    
    # Calculate hour angle
    ha = np.arctan2(-np.sin(azim), np.tan(elev)*np.cos(lat) - np.cos(azim)*np.sin(lat))

    # Calculate Local Sidereal Time
    lst = np.radians(JD2LST(jd, np.degrees(lon))[0])
    
    # Calculate right ascension
    ra = (lst - ha)%(2*np.pi)

    # Calculate declination
    dec = np.arcsin(np.sin(lat)*np.sin(elev) + np.cos(lat)*np.cos(elev)*np.cos(azim))

    return np.degrees(ra), np.degrees(dec)

def raDec2AltAz(ra, dec, jd, lat, lon):
    """ Convert right ascension and declination to azimuth (+east of sue north) and altitude. 
    Arguments:
        ra: [float] right ascension in degrees
        dec: [float] declination in degrees
        jd: [float] Julian date
        lat: [float] latitude in degrees
        lon: [float] longitude in degrees
    Return:
        (azim, elev): [tuple]
            azim: [float] azimuth (+east of due north) in degrees
            elev: [float] elevation above horizon in degrees
        """

    ra = np.radians(ra)
    dec = np.radians(dec)
    lat = np.radians(lat)
    lon = np.radians(lon)

    # Calculate Local Sidereal Time
    lst = np.radians(JD2LST(jd, np.degrees(lon))[0])

    # Calculate the hour angle
    ha = lst - ra

    # Constrain the hour angle to [-pi, pi] range
    ha = (ha + np.pi)%(2*np.pi) - np.pi

    # Calculate the azimuth
    azim = np.pi + np.arctan2(np.sin(ha), np.cos(ha)*np.sin(lat) - np.tan(dec)*np.cos(lat))

    # Calculate the sine of elevation
    sin_elev = np.sin(lat)*np.sin(dec) + np.cos(lat)*np.cos(dec)*np.cos(ha)

    # Wrap the sine of elevation in the [-1, +1] range
    sin_elev = (sin_elev + 1)%2 - 1

    elev = np.arcsin(sin_elev)

    return np.degrees(azim), np.degrees(elev)

# use:
# (ra, dec)    = altAz2RADec(azim, elev, datetime2JD(), lat, lon)
# (azim, elev) = raDec2AltAz(azim, elev, datetime2JD(), lat, lon)


# Vectorize the raDec2AltAz function so it can take numpy arrays for: ra, dec, jd
raDec2AltAz_vect = np.vectorize(raDec2AltAz, excluded=['lat', 'lon'])





### Precession ###

def equatorialCoordPrecession(start_epoch, final_epoch, ra, dec):
    """ Corrects Right Ascension and Declination from one epoch to another, taking only precession into 
        account.
        Implemented from: Jean Meeus - Astronomical Algorithms, 2nd edition, pages 134-135
    @param start_epoch: [float] Julian date of the starting epoch
    @param final_epoch: [float] Julian date of the final epoch
    @param ra: [float] non-corrected right ascension in degrees
    @param dec: [float] non-corrected declination in degrees
    @return (ra, dec): [tuple of floats] precessed equatorial coordinates in degrees
    """

    ra = math.radians(ra)
    dec = math.radians(dec)

    T = (start_epoch - 2451545) / 36525.0
    t = (final_epoch - start_epoch) / 36525.0

    # Calculate correction parameters
    zeta  = ((2306.2181 + 1.39656*T - 0.000139*T**2)*t + (0.30188 - 0.000344*T)*t**2 + 0.017998*t**3)/3600
    z     = ((2306.2181 + 1.39656*T - 0.000139*T**2)*t + (1.09468 + 0.000066*T)*t**2 + 0.018203*t**3)/3600
    theta = ((2004.3109 - 0.85330*T - 0.000217*T**2)*t - (0.42665 + 0.000217*T)*t**2 - 0.041833*t**3)/3600

    # Convert parameters to radians
    zeta, z, theta = map(math.radians, (zeta, z, theta))

    # Calculate the next set of parameters
    A = math.cos(dec) * math.sin(ra + zeta)
    B = math.cos(theta)*math.cos(dec)*math.cos(ra + zeta) - math.sin(theta)*math.sin(dec)
    C = math.sin(theta)*math.cos(dec)*math.cos(ra + zeta) + math.cos(theta)*math.sin(dec)

    # Calculate right ascension
    ra_corr = math.atan2(A, B) + z

    # Calculate declination (apply a different equation if close to the pole, closer then 0.5 degrees)
    if (math.pi/2 - abs(dec)) < math.radians(0.5):
        dec_corr = math.acos(math.sqrt(A**2 + B**2))
    else:
        dec_corr = math.asin(C)

    temp_ra = math.degrees(ra_corr)    
    if temp_ra < 0:
        temp_ra += 360.

    return temp_ra, math.degrees(dec_corr)



# Calculate UFO-style ra and dec by fitting a great circle
def ufo_ra_dec_alt_az(ttt):
    
    # Compute times of first and last points
    no_lines = len(ttt['datetime'])
    try:
        dt1 = datetime.strptime(str(ttt['datetime'][0]),ISO_FORMAT)
        dt2 = datetime.strptime(str(ttt['datetime'][no_lines - 1]),ISO_FORMAT)
    except:
        dt1 = datetime.strptime(str(ttt['datetime'][0]),"%Y-%m-%d %H:%M:%S.%f")
        dt2 = datetime.strptime(str(ttt['datetime'][no_lines - 1]),"%Y-%m-%d %H:%M:%S.%f")
    #JD = datetime2JD(dt1)

        
    ### Fit a great circle to Az/Alt measurements and compute model beg/end RA and Dec ###

    # Convert the measurement Az/Alt to cartesian coordinates
    # NOTE: All values that are used for Great Circle computation are:
    #   theta - the zenith angle (90 deg - altitude)
    #   phi - azimuth +N of due E, which is (90 deg - azim)
    azim = ttt['azimuth']
    elev = ttt['altitude']
    x, y, z = polarToCartesian(np.radians((90 - azim)%360), np.radians(90 - elev))

    # Fit a great circle
    C, theta0, phi0 = fitGreatCircle(x, y, z)

    # Get the first point on the great circle
    phase1 = greatCirclePhase(np.radians(90 - elev[0]), np.radians((90 - azim[0])%360), \
        theta0, phi0)
    alt1, azim1 = cartesianToPolar(*greatCircle(phase1, theta0, phi0))
    alt1 = 90 - np.degrees(alt1)
    azim1 = (90 - np.degrees(azim1))%360

    # Get the last point on the great circle
    phase2 = greatCirclePhase(np.radians(90 - elev[-1]), np.radians((90 - azim[-1])%360),\
        theta0, phi0)
    aa, bb, cc = greatCircle(phase2, theta0, phi0)
    alt2, azim2 = cartesianToPolar(aa, bb, cc)
    alt2 = 90 - np.degrees(alt2)
    azim2 = (90 - np.degrees(azim2))%360

    # Compute RA/Dec from Alt/Az
    obs_latitude = float(ttt.meta['obs_latitude'])
    obs_longitude = float(ttt.meta['obs_longitude'])
    ra1, dec1 = altAz2RADec(azim1, alt1, datetime2JD(dt1), obs_latitude, obs_longitude)
    ra2, dec2 = altAz2RADec(azim2, alt2, datetime2JD(dt2), obs_latitude, obs_longitude)


    return(float(alt1), float(alt2), float(azim1), float(azim2), float(ra1), float(ra2), float(dec1), float(dec2));


""" Fitting a great circle to points in the Cartesian coordinates system. """

# The MIT License

# Copyright (c) 2017, Denis Vida

# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:

# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.

# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN
# THE SOFTWARE.

#from __future__ import print_function, division, absolute_import

import scipy.linalg
import scipy.optimize


def greatCirclePhase(theta, phi, theta0, phi0):
    """ Find the phase angle of the point closest to the given point on the great circle. 
    
    Arguments:
        theta: [float] Inclination of the point under consideration (radians).
        phi: [float] Nodal angle of the point (radians).
        theta0: [float] Inclination of the great circle (radians).
        phi0: [float] Nodal angle of the great circle (radians).
    Return:
        [float] Phase angle on the great circle of the point under consideration (radians).
    """

    def _pointDist(x):
        """ Calculates the Cartesian distance from a point defined in polar coordinates, and a point on
            a great circle. """
        
        # Convert the pick to Cartesian coordinates
        point = polarToCartesian(phi, theta)

        # Get the point on the great circle
        circle = greatCircle(x, theta0, phi0)

        # Return the distance from the pick to the great circle
        return np.sqrt((point[0] - circle[0])**2 + (point[1] - circle[1])**2 + (point[2] - circle[2])**2)

    # Find the phase angle on the great circle which corresponds to the pick
    res = scipy.optimize.minimize(_pointDist, 0)

    return res.x



def greatCircle(t, theta0, phi0):
    """ 
    Calculates the point on a great circle defined my theta0 and phi0 in Cartesian coordinates. 
    
    Sources:
        - http://demonstrations.wolfram.com/ParametricEquationOfACircleIn3D/
    Arguments:
        t: [float or 1D ndarray] phase angle of the point in the great circle
        theta0: [float] Inclination of the great circle (radians).
        phi0: [float] Nodal angle of the great circle (radians).
    Return:
        [tuple or 2D ndarray] a tuple of (X, Y, Z) coordinates in 3D space (becomes a 2D ndarray if the input
            parameter t is also a ndarray)
    """


    # Calculate individual cartesian components of the great circle points
    x = -np.cos(t)*np.sin(phi0) + np.sin(t)*np.cos(theta0)*np.cos(phi0)
    y =  np.cos(t)*np.cos(phi0) + np.sin(t)*np.cos(theta0)*np.sin(phi0)
    z =  np.sin(t)*np.sin(theta0)

    return x, y, z




def fitGreatCircle(x, y, z):
    """ Fits a great circle to points in 3D space. 
    Arguments:
        x: [float] X coordiantes of points on the great circle.
        y: [float] Y coordiantes of points on the great circle.
        z: [float] Z coordiantes of points on the great circle.
    Return: 
        X, theta0, phi0: [tuple of floats] Great circle parameters.
    """

    # Add (0, 0, 0) to the data, as the great circle should go through the origin
    x = np.append(x, 0)
    y = np.append(y, 0)
    z = np.append(z, 0)

    # Fit a linear plane through the data points
    A = np.c_[x, y, np.ones(x.shape[0])]
    C,_,_,_ = scipy.linalg.lstsq(A, z)

    # Calculate the great circle parameters
    z2 = C[0]**2 + C[1]**2

    theta0 = np.arcsin(z2/np.sqrt(z2 + z2**2))
    phi0 = np.arctan2(C[1], C[0])

    return C, theta0, phi0
    


def cartesianToPolar(x, y, z):
    """ Converts 3D cartesian coordinates to polar coordinates. 
    Arguments:
        x: [float] Px coordinate.
        y: [float] Py coordinate.
        z: [float] Pz coordinate.
    Return:
        (theta, phi): [float] Polar angles in radians (inclination, azimuth).
    """

    theta = np.arccos(z)
    phi = np.arctan2(y, x)

    return theta, phi



def polarToCartesian(theta, phi):
    """ Converts 3D spherical coordinates to 3D cartesian coordinates. 
    Arguments:
        theta: [float] Inclination in radians.
        phi: [float] Azimuth angle in radians.
    Return:
        (x, y, z): [tuple of floats] Coordinates of the point in 3D cartiesian coordinates.
    """


    x = np.sin(phi)*np.cos(theta)
    y = np.sin(phi)*np.sin(theta)
    z = np.cos(phi)

    return x, y, z
  

# Utility functions

In [ ]:
# general-purpose file-handling or numerical functions

# GCD (or Highest Common Factor) of two numbers 
def find_gcd(x, y): 
    while(y): 
        x, y = y, x % y 
    return x 
 
    
# GCD (or Highest Common Factor) of integers in an array 
def array_gcd(l): 
    len_array = len(l)
    if len_array == 0:
        return(0);
    elif len_array == 1:
        return(l[0]);
    elif len_array == 2:
        return(find_gcd(l[0],l[1]));
    else:
        gcd = find_gcd(l[0],l[1])
        for i in range(2, len_array): 
            gcd = find_gcd(gcd, l[i]) 
        return(gcd);  

    
#get the number of seconds since start_time     
def get_secs(ttt_date,start_time):     
    head_days = Time(ttt_date)
    head_days -= start_time
    return(float(str(head_days))*24*60*60);    


# define city fuction
#def getcity(latlong):
#    locator = Nominatim(user_agent="contact@ukfall.org.uk",timeout = 10)
#    rgeocode = RateLimiter(locator.reverse,min_delay_seconds = 0.001)
#    try:
#        location = rgeocode(latlong)
#        di = dict(location.raw)
#        if 'city' in di.keys():
#            city = di['city']
#        elif 'village' in di.keys():
#            city = di['village']
#        elif 'town' in di.keys():
#            city = di['town']
#        else:
#            city = 'no city or town'
#    except:
#        city = 'city not found'
#    return city # or return location.raw to see all the data


def zipfilename(ttt_list, out_type):
# returns a name for the zipped output file, e.g. 2020-12-31_UFO_EastBarnet.zip
# ttt_list is a list of astropy tables, out_type is a string decribing the type of data written.

    ttt = ttt_list[0]
    location = ttt.meta['location']
    no_meteors = len(ttt_list)
    st = str(ttt['datetime'][0])

    for k in range(no_meteors):
        if ttt_list[k].meta['location'] != location:
            location = 'MultiLocation'
    
    initial_file = st[0:10] + '_' + out_type + '_' + location[0:15] + '.zip'
    # print('initial_file = ',initial_file)

    return initial_file 
    
   
def outfilename(ttt_list, out_type, source, is_main, num_days, i):
# returns a name for the output file, e.g. 2020-12-31_UFO_EastBarnet.zip
# ttt is an astropy table, out_type is a string decribing the type of data written.
# source is a string describing where the data came from.
# is_main is True for the main data file, False for the ancillary (e.g. FRIPON location 
# or UFO csv summary) file. 
# "num_days" is used for the UFO CSV file name or is the the file number in DFN files.
# 'i' is the index of the meteor.


    ttt = ttt_list[i]
    location = ttt.meta['location']
    telescope = ttt.meta['telescope']
    if out_type == 'UFO' and not is_main:
        # check the name of the csv file
        no_meteors = len(ttt_list)
        for k in range(no_meteors):
            if ttt_list[k].meta['location'] != location:
                location = 'MultiLocation'
            if ttt_list[k].meta['telescope'] != telescope:
                telescope = 'MultiLocation'
    location = location.replace(" ", "_")[0:15]
    telescope = telescope.replace(" ", "_")[0:15]
    if location == telescope:
        telescope = ''
    else:    
        telescope = '_' + telescope

        
    if out_type == 'STD':
        # Standard output, in form 2020-05-11T22_41_00_RMS_UK0002.ecsv.  
        st = str(ttt['datetime'][0])  
        output_file = st[0:19].replace(":", "_")
        output_file += '_' + source + '_' 
        output_file += ttt.meta['camera_id'].replace(" ", "_")[0:15]
        output_file += '.ecsv' 
    
    elif out_type == 'DFN' :
        # Desert Fireball Network output
        st = str(ttt['datetime'][0])  
        output_file = str(num_days).zfill(3) + "_" + st[0:10] + '_'
        output_file += st[11:13] + st[14:16] + st[17:19] + '_'
        output_file += location + telescope + '.ecsv' 
    
    elif out_type == 'UFO':
        # UFOAnalyzer output files
        if is_main :
            # write the A.XML file in format : "M20200601_220346_EastBarnet_NEA.XML"
            output_file = 'M' + isoStr(ttt['datetime'][0]).strftime('%Y%m%d_%H%M%S_') + "00_"
            output_file += location + '_' + ttt.meta['camera_id'][0:2] + '_A.XML'
        else:    
            # name of the CSV file, e.g. 20201231_23_188_EastBarnet_NW.csv,
            output_file = isoStr(ttt['datetime'][0]).strftime('%Y%m%d_%H_') + str(num_days).zfill(3) + "_"
            output_file += location + '.csv'
    
    elif out_type == 'FRIPON':
        # example 20200103T170201_UT_FRNO01_SJ.met
        st = str(ttt['datetime'][0])  #e.g. 2020-01-03T17:02:01.885
        output_file = st[0:4] + st[5:7]+ st[8:13]
        output_file += st[14:16] + st[17:19] + '_UT' + telescope
        if is_main :
            output_file += '.met'
        else:    
            output_file += '_location.txt'
            
    else:
        # A CSV file readable by Excel, or an ASC output file,
        # plus a catch-all if file type unknown
        st = str(ttt['datetime'][0]) #e.g. 2020-01-03T17:02:01.885
        output_file =  st[0:10] + '_'
        output_file += st[11:13] + st[14:16] + st[17:19] + '_'
        if out_type == 'ASC':
            output_file += location + telescope + '.json' 
        else:    
            output_file += location + telescope + '.csv' 
    
    return output_file 


def std_timeshift(ttt,sec):
    # changes all of the dates in a standard table to make them earlier by a number of seconds equal to 'sec'    
    
    ttt.meta['isodate_start_obs'] = change_str_time(ttt.meta['isodate_start_obs'],sec) 
    ttt.meta['isodate_calib'] = change_str_time(ttt.meta['isodate_calib'],sec) 
    nlines = len(ttt['datetime'])    
    for i in range(nlines):
        ttt['datetime'][i] = change_str_time(ttt['datetime'][i],sec) 
    return ttt 


def change_str_time(in_str, sec):
    # Takes an ISO datetime string, calculates a time earlier by 'sec', returning an ISO datetime string.
    in_time = Time(in_str)
    new_time = in_time + timedelta(seconds=-sec)
    out_str = str(new_time)
    return out_str

## Main program

In [ ]:
print("\nstarting program\n")

file_read_types = (("all files","*.*"),("Standard","*.ECSV"),("UFOAnalyzer","*A.XML"),\
        ("UKFN/DFN","*.ECSV"),("SCAMP/FRIPON","*.MET"),("SCAMP/FRIPON","*.ZIP"),("RMS/CAMS","FTP*.txt"),\
        ("RMS/AllSkyCams","*.json"),("MetRec","*.inf"))
_fname = filedialog.askopenfilename(multiple=False,title = "Select file to read",filetypes = file_read_types)

if _fname == None or len(_fname) < 3:
    sys.exit('User did not choose a file to open')

lname = _fname.lower()
print("Input data file chosen is: ",lname) 
initial_dir, file_name = os.path.split(_fname)


if lname.endswith(".ecsv"):
    # input is STANDARD or DFN
    _obs_table = ascii.read(_fname, delimiter=',') 
    print(_obs_table)
    
    DFN_true = False
    for key_name in _obs_table.meta.keys(): 
        if 'event_codename' in key_name: 
            DFN_true = True
            
    if  DFN_true :
        print("DFN/UKFN format being read")
        ttt_list, meteor_count = dfn_to_std(_obs_table)
        source = 'DFN'
    else:    
        print("standard format being read")
        ttt_list = [_obs_table]
        meteor_count = 1
        source = 'STD'
        
    
elif lname.endswith("a.xml"):
    # input is UFOAnalyzer.  
    print("UFO format being read")
    with open(_fname) as fd:
        _obs_dic=xmltodict.parse(fd.read())
    ttt_list, meteor_count = ufo_to_std(_obs_dic)
    source = 'UFO'

    
elif lname.endswith(".met"):
    # input is FRIPON/SCAMP
    print("FRIPON/SCAMP format being read")
    source = 'FRIPON'
    loc_table, no_stations = get_fripon_stations()
    _obs_table = Table.read(_fname, format='ascii.sextractor')
    ttt_list, meteor_count = fripon_to_std(_fname,_obs_table, loc_table, no_stations)

    
elif lname.endswith(".zip"):
    # input is a FRIPON zipped results file usually containing multiple .met files
    print("FRIPON zipped format being read")

    ttt_list = []
    meteor_count = 0
    source = 'FRIPON'
    loc_table, no_stations = get_fripon_stations()
    
    # get the list of .met files
    with ZipFile(lname, 'r') as zip: 
        for info in zip.infolist(): 
            if info.filename.endswith(".met"):
                # extract, read and delete each ".met" file
                z_fname = zip.extract(info.filename)
                _obs_table = Table.read(z_fname, format='ascii.sextractor')
                os.remove(z_fname)
                ttt_list2, meteor_count2 = fripon_to_std(z_fname,_obs_table, loc_table, no_stations)
                if meteor_count2 > 0:
                    meteor_count += meteor_count2
                    ttt_list += ttt_list2

     
elif lname.endswith(".txt"):
    # input is RMS or CAMS 
    meteor_text = open(_fname).read()
    ttt_list = [] 

    # now check whether there is a platpars file in the same folder, i.e. input is RMS
    _camera_file_path = os.path.join(initial_dir, 'platepars_all_recalibrated.json')
    if not (os.path.exists( _camera_file_path)):
        # no platpars file, so look for a CAL*.txt file 
        cal_files = []
        all_files = os.listdir(initial_dir)
        for file_name in all_files :
            fname_low = file_name.lower() 
            if (fname_low.startswith('cal') and fname_low.endswith('.txt')): 
                cal_files.append(file_name)
        if(len(cal_files) == 1 ):
            # CAMS, one CAL file found
            _camera_file_path = os.path.join(initial_dir, cal_files[0])
        elif(len(cal_files) > 1 ):
            # CAMS, multiple CAL files found
            file_read_types = (("CAMS, CAL*.txt","*.txt"))
            _camera_file_path = filedialog.askopenfilename(multiple=False,initialdir = initial_dir, initialfile=cal_files[0],title = "Select one CAMS camera data file", filetypes = file_read_types)
        else:       
            # no camera files found.  Ask the user for the RMS or CAMS camera metadata file name
            file_read_types = (("all files","*.*"),("CAMS, CAL*.txt","*.txt"),("RMS, *.JSON","*.json"))
            _camera_file_path = filedialog.askopenfilename(multiple=False,initialdir = initial_dir, title = "Select an RMS or CAMS camera data file", filetypes = file_read_types)
    
    if not _camera_file_path:
        sys.exit("Camera Config not specified, exiting")
    if not (os.path.exists( _camera_file_path)):
        sys.exit("Camera Config not found, exiting")
    _camera_lfile = _camera_file_path.lower()
    print("Cam Data Path : ",_camera_lfile)

    if _camera_lfile.endswith(".json"):
        # Input is RMS
        print("RMS format being read")
        rms_camera_data = rms_camera_json(_camera_file_path)
        ttt_list, meteor_count =  rms_to_std(meteor_text,  rms_camera_data)
        source = 'RMS'
    elif _camera_lfile.endswith(".txt"):
        # Input is CAMS
        print("CAMS format being read")
        cams_camera_data = cams_camera_txt( _camera_file_path )
        ttt_list, meteor_count = cams_to_std(meteor_text, cams_camera_data)
        source = 'CAMS'
    else:
        sys.exit("Camera file not supported. Please supply a platepars JSON file (.json) for RMS, or a CAL TXT file (.txt) for CAMS")

        
elif lname.endswith(".json"):
    _json_str = open(_fname).read()
    json_data = json.loads(_json_str)
    if 'centroids' in json_data :  
        # This is an RMS format
        print("RMS .json format being read")
        source = 'RMS'
        ttt_list, meteor_count = rms_json_to_std(json_data,lname)
        
    else:     
        # input is AllSkyCams
        print("AllSkyCams format being read")
        ttt_list, meteor_count = allskycams_to_std(json_data,lname)
        source = 'ASC'
    
    
elif lname.endswith(".inf"):
    # Input is MetRec
    # now look for the .log file    
    print("MetRec format being read")
    log_files = []
    all_files = os.listdir(initial_dir)
    for file_name in all_files :
        fname_low = file_name.lower() 
        if (fname_low.endswith('.log') and not(fname_low.startswith('mrg') or fname_low.startswith('states'))): 
            log_files.append(file_name)
    if(len(log_files) == 1 ):
        # one .log file found
        _log_file_path = os.path.join(initial_dir, log_files[0])
    else:
        # Ask the user to choose the log file
        file_read_types = (("MetRec log file", "*.log"))
        _log_file_path = filedialog.askopenfilename(multiple=False,initialdir = initial_dir, title = "Select camera data file", filetypes = file_read_types)
    
    print('MetRec log file used = ',_log_file_path)
    inf = MetRecInfFile(_fname)
    log = MetRecLogFile(_log_file_path)
    
    ttt_list, meteor_count = metrec_to_standard(inf, log)
    source = 'MetRec'

    
    
if meteor_count == 0:
    sys.exit("No meteors detected - check file is correct")
else:
    ttt = ttt_list[0]


In [ ]:
print("Number of meteors read: ", meteor_count)
output_type = int(input("\nChoose output format: 1=Global Fireball Exchange (GFE), 2=UFO, 3=DFN/UKFN, 4=FRIPON, 5=AllSkyCams, 9=ExcelCSV :"))
print("You entered " + str(output_type))

# must make a 'file-like object' to allow astropy to write to zip ( ie: must have file.write(datastring) )
class AstropyWriteZipFile:
    def __init__(self, out_zip, out_file):
        self.zip  = out_zip
        self.at   = out_file
        self.done = False
            
    def write(self, data):
        if not self.done:
            self.zip.writestr(self.at, data)
            self.done = True
            
def isoStr(iso_datetime_string):
     return datetime.strptime(iso_datetime_string, ISO_FORMAT)

    
# Write file(s) depending on input
if (output_type<1 or output_type> 5) and not output_type==9:
    sys.exit('Not valid input - it needed to be 1, 2, 3, 4, 5 or 9')

elif output_type == 1 or output_type == 3:
    #write Standard or DFN format
    if output_type == 3:
        out_type = 'DFN'
    else:    
        out_type = 'STD'
    
    if meteor_count > 1:
        zip_file_init = zipfilename(ttt_list, out_type)
        out_name = filedialog.asksaveasfilename(initialdir=initial_dir,initialfile=zip_file_init,title = "Save file")
        # zipset = {}
        if out_name:
            output_zip = ZipFile(out_name, mode='w')
            for i in range (meteor_count):
                if output_type == 3:
                    ttt = std_to_dfn(ttt_list[i])
                else:     
                    ttt = ttt_list[i]
                out_name2 = outfilename(ttt_list, out_type, source, True, 0, i)
                ascii.write(ttt, AstropyWriteZipFile(output_zip, out_name2), format='ecsv', delimiter=',')
            output_zip.close()
            print("Zip file written: ", out_name)
    else: 
        # write output to a single file
        initial_file = outfilename(ttt_list, out_type, source, True, 0, 0)
        out_name = filedialog.asksaveasfilename(initialdir=initial_dir,initialfile=initial_file,title = "Save file")
        if out_name :
            if output_type == 3:
                ttt = std_to_dfn(ttt_list[0])
            else:     
                ttt = ttt_list[0]
            ttt.write(out_name,overwrite=True, format='ascii.ecsv', delimiter=',')
            print("Data file written: ", out_name)

            
elif output_type == 2:
    # UFOAnalyzer output - always written to a zip file    
    zip_file_init = zipfilename(ttt_list, 'UFO')
    zip_file_name = filedialog.asksaveasfilename(initialdir = initial_dir,initialfile=zip_file_init,title = "Select file",defaultextension = '.csv')
    if zip_file_name :
        output_zip     = ZipFile(zip_file_name, mode='w')
        output_csv_str = ""
        for i in range(len(ttt_list)):
            ttt = ttt_list[i]
            #converts to 2 strings - the XML file and one line from the CSV file
            ufo_xml_data, ufo_csv_line = std_to_ufo(ttt)
            out_name2 = outfilename(ttt_list, 'UFO', source, True, 0, i)
            output_zip.writestr(out_name2, ufo_xml_data)
            if i == 0:
                output_csv_string = ufo_csv_line
            else:
                output_csv_string += '\n' + ufo_csv_line.split('\n')[1]
                
        #difference in days:
        num_days = ( isoStr(ttt_list[-1]['datetime'][0]) - isoStr(ttt_list[0]['datetime'][0]) ).days
        out_csv_file = outfilename(ttt_list, 'UFO', source, False, num_days, 0)
        output_zip.writestr(out_csv_file, output_csv_string)
        output_zip.close()
        print("Zip file written: ", zip_file_name)
        
        
elif output_type == 4:
    # write a file in FRIPON/SCAMP format
    zip_file_init = zipfilename(ttt_list, 'FRIPON')
    zip_file_name = filedialog.asksaveasfilename(initialdir = initial_dir,initialfile=zip_file_init,title = "Select file",defaultextension = '.csv')
        
    if zip_file_name :
        output_zip     = ZipFile(zip_file_name, mode='w')
        for i in range(len(ttt_list)):
            ttt = ttt_list[i]
            ttt2 = std_to_fripon(ttt)
            fri_str, loc_str = fripon_write(ttt2)
            out_name2 = outfilename(ttt_list, 'FRIPON', source, True, 0, i)
            output_zip.writestr(out_name2, fri_str)                
            out_name2 = outfilename(ttt_list, 'FRIPON', source, False, 0, i)
            output_zip.writestr(out_name2, loc_str)                
        output_zip.close()
        print("Zip file written: ", zip_file_name)
 


elif output_type == 5:
    #write AllSkyCams format
    if meteor_count > 1:
        zip_file_init = zipfilename(ttt_list, 'ASC')
        zip_file_name = filedialog.asksaveasfilename(initialdir = initial_dir,initialfile=zip_file_init,title = "Select file",defaultextension = '.csv')
        
        if zip_file_name :
            output_zip     = ZipFile(zip_file_name, mode='w')
            for i in range(len(ttt_list)):
                ttt = ttt_list[i]
                json_str = std_to_allskycams(ttt)
                out_name2 = outfilename(ttt_list, 'ASC', source, True, 0, i)
                output_zip.writestr(out_name2, json_str)                
            output_zip.close()
            print("Zip file written: ", zip_file_name)
    else: 
        # write AllSkyCams data to a single file
        ttt = ttt_list[0]
        initial_file = outfilename(ttt_list, 'ASC', source, True, 0, 0)
        out_name = filedialog.asksaveasfilename(initialdir=initial_dir,initialfile=initial_file,title = "Save file")
        if out_name :
            # write json_str to a file called out_name
            json_str = std_to_allskycams(ttt)
            out_file = open(out_name, "w")
            out_file.write(json_str)
            out_file.flush()
            out_file.close()
            print("Data file written: ", out_name)

            
elif output_type == 9:
    #write Excel csv format
    if meteor_count > 1:
        zip_file_init = zipfilename(ttt_list, 'CSV')
        zip_file_name = filedialog.asksaveasfilename(initialdir = initial_dir,initialfile=zip_file_init,title = "Select file",defaultextension = '.csv')
        
        if zip_file_name :
            output_zip     = ZipFile(zip_file_name, mode='w')
            for i in range(len(ttt_list)):
                ttt = ttt_list[i]
                csv_str = std_to_csv(ttt)
                out_name2 = outfilename(ttt_list, 'CSV', source, True, 0, i)
                output_zip.writestr(out_name2, csv_str)                
            output_zip.close()
            print("Zip file written: ", zip_file_name)
    else: 
        # write Excel csv data to a single file
        ttt = ttt_list[0]
        initial_file = outfilename(ttt_list, 'CSV', source, True, 0, 0)
        out_name = filedialog.asksaveasfilename(initialdir=initial_dir,initialfile=initial_file,title = "Save file")
        if out_name :
            # write csv_str to a file called out_name
            csv_str = std_to_csv(ttt)
            out_file = open(out_name, "w")
            out_file.write(csv_str)
            out_file.flush()
            out_file.close()
            print("Data file written: ", out_name)


else:        
     print('Invalid output type chosen (',output_type,')')

        
print('finished')  
